<a href="https://www.kaggle.com/code/cheikhsadibousidibe/etbet-lstm?scriptVersionId=264590550" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# generation of dataset for finetuning

## importations

In [ ]:
!pip install scapy

In [ ]:
import os
import json
import random
import binascii
import pandas as pd
import numpy as np
import tqdm
import scapy.all as scapy
from sklearn.model_selection import train_test_split
from scapy.all import rdpcap, wrpcap,PcapReader
from collections import defaultdict,Counter
from concurrent.futures import ProcessPoolExecutor, as_completed
from multiprocessing import Pool, cpu_count
from scapy.layers.inet import IP, TCP, UDP
import hashlib
import time
import psutil
import gc
import csv
# Scapy imports
from scapy.all import rdpcap, wrpcap
from scapy.layers.inet import IP, TCP, UDP, ICMP
from scapy.layers.l2 import ARP
from scapy.layers.dhcp import DHCP
from scapy.layers.dns import DNS
from scapy.layers.inet6 import IPv6
from pathlib import Path
import shutil
import re

In [ ]:
for dirpath,dirname, file in os.walk("/kaggle/input/pcap-data/ustf_tfc"):
    print(dirpath)
    print(dirname)
    print(file)

In [ ]:
#reorganizer ISCXVPN files 
def organize_pcap_files(input_directories, output_directory, final_labels=None, file_label_map=None):
    """
    Organize PCAP files from input directories into labeled subdirectories.
    
    Args:
        input_directories (list): List of input directory paths containing PCAP files
        output_directory (str): Output directory where organized files will be placed
        final_labels (list): Target label categories (default: ["browsing", "chat", "email", "ftp", "p2p", "streaming", "voip"])
        file_label_map (dict): Explicit mapping of filenames to labels
    
    Returns:
        dict: Summary of files organized by category
    """
    if final_labels is None:
        final_labels = ["browsing", "chat", "email", "ftp", "p2p", "streaming", "voip"]
    
    # Use provided file label mapping or default mapping
    if file_label_map is None:
        file_label_map = {
            # --- voip ---
            "vpn_facebook_audio2.pcap": "voip",
            "vpn_hangouts_audio2.pcap": "voip",
            "vpn_hangouts_audio1.pcap": "voip",
            "vpn_skype_audio2.pcap": "voip",
            "vpn_skype_audio1.pcap": "voip",
            "vpn_voipbuster1a.pcap": "voip",
            "vpn_voipbuster1b.pcap": "voip",
            # --- chat ---
            "vpn_hangouts_chat1a.pcap": "chat",
            "vpn_hangouts_chat1b.pcap": "chat",
            "vpn_facebook_chat1a.pcap": "chat",
            "vpn_facebook_chat1b.pcap": "chat",
            "vpn_aim_chat1a.pcap": "chat",
            "vpn_aim_chat1b.pcap": "chat",
            "vpn_skype_chat1a.pcap": "chat",
            "vpn_skype_chat1b.pcap": "chat",
            "vpn_icq_chat1a.pcap": "chat",
            "vpn_icq_chat1b.pcap": "chat",
            # --- email ---
            "vpn_email2a.pcap": "email",
            "vpn_email2b.pcap": "email",
            # --- ftp ---
            "vpn_ftps_A.pcap": "ftp",
            "vpn_ftps_B.pcap": "ftp",
            "vpn_sftp_A.pcap": "ftp",
            "vpn_sftp_B.pcap": "ftp",
            "vpn_skype_files1a.pcap": "ftp",
            "vpn_skype_files1b.pcap": "ftp",
            # --- streaming ---
            "vpn_vimeo_A.pcap": "streaming",
            "vpn_vimeo_B.pcap": "streaming",
            "vpn_youtube_A.pcap": "streaming",
            "vpn_spotify_A.pcap": "streaming",
            "vpn_netflix_A.pcap": "streaming",
            # --- p2p ---
            "vpn_bittorrent.pcap": "p2p"
        }
    
    # Initialize results
    results = defaultdict(list)
    skipped_files = []
    
    print(f"🚀 Starting PCAP file organization")
    print(f"📁 Target labels: {final_labels}")
    print(f"📂 Output directory: {output_directory}")
    
    # Create output directory and subdirectories
    os.makedirs(output_directory, exist_ok=True)
    for label in final_labels:
        label_dir = os.path.join(output_directory, label)
        os.makedirs(label_dir, exist_ok=True)
        print(f"📁 Created directory: {label_dir}")
    
    # Process each input directory
    total_processed = 0
    total_copied = 0
    
    for input_dir in input_directories:
        if not os.path.exists(input_dir):
            print(f"⚠️ Input directory does not exist: {input_dir}")
            continue
            
        print(f"\n📂 Processing directory: {input_dir}")
        
        # Get all PCAP files in the directory
        pcap_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.pcap', '.pcapng'))]
        print(f"📄 Found {len(pcap_files)} PCAP files")
        
        for filename in pcap_files:
            total_processed += 1
            source_path = os.path.join(input_dir, filename)
            
            # Look up label directly from the mapping
            matched_label = file_label_map.get(filename)
            
            if matched_label and matched_label in final_labels:
                # Copy file to appropriate subdirectory
                dest_dir = os.path.join(output_directory, matched_label)
                dest_path = os.path.join(dest_dir, filename)
                
                try:
                    shutil.copy2(source_path, dest_path)
                    results[matched_label].append(filename)
                    total_copied += 1
                    print(f"✅ {filename} → {matched_label}/")
                except Exception as e:
                    print(f"❌ Error copying {filename}: {e}")
                    skipped_files.append((filename, f"Copy error: {e}"))
            else:
                skipped_files.append((filename, "No matching label found"))
                print(f"⏭️ Skipped: {filename} (no clear category match)")
    
    # Print summary
    print(f"\n{'='*60}")
    print(f"📊 ORGANIZATION SUMMARY")
    print(f"{'='*60}")
    print(f"📈 Total files processed: {total_processed}")
    print(f"✅ Total files copied: {total_copied}")
    print(f"⏭️ Total files skipped: {len(skipped_files)}")
    
    print(f"\n📁 Files by category:")
    for label in final_labels:
        count = len(results[label])
        print(f"  {label}: {count} files")
        if count > 0:
            # Show first few files as examples
            examples = results[label][:3]
            for example in examples:
                print(f"    📄 {example}")
            if len(results[label]) > 3:
                print(f"    ... and {len(results[label]) - 3} more")
    
    if skipped_files:
        print(f"\n⏭️ Skipped files:")
        for filename, reason in skipped_files[:10]:  # Show first 10
            print(f"  📄 {filename}: {reason}")
        if len(skipped_files) > 10:
            print(f"  ... and {len(skipped_files) - 10} more")
    
    return dict(results)

def get_file_category_mapping():
    """
    Return the exact mapping used for categorizing files.
    This helps understand how files are being classified.
    """
    return {
        # --- voip ---
        "vpn_facebook_audio2.pcap": "voip",
        "vpn_hangouts_audio2.pcap": "voip",
        "vpn_hangouts_audio1.pcap": "voip",
        "vpn_skype_audio2.pcap": "voip",
        "vpn_skype_audio1.pcap": "voip",
        "vpn_voipbuster1a.pcap": "voip",
        "vpn_voipbuster1b.pcap": "voip",
        # --- chat ---
        "vpn_hangouts_chat1a.pcap": "chat",
        "vpn_hangouts_chat1b.pcap": "chat",
        "vpn_facebook_chat1a.pcap": "chat",
        "vpn_facebook_chat1b.pcap": "chat",
        "vpn_aim_chat1a.pcap": "chat",
        "vpn_aim_chat1b.pcap": "chat",
        "vpn_skype_chat1a.pcap": "chat",
        "vpn_skype_chat1b.pcap": "chat",
        "vpn_icq_chat1a.pcap": "chat",
        "vpn_icq_chat1b.pcap": "chat",
        # --- email ---
        "vpn_email2a.pcap": "email",
        "vpn_email2b.pcap": "email",
        # --- ftp ---
        "vpn_ftps_A.pcap": "ftp",
        "vpn_ftps_B.pcap": "ftp",
        "vpn_sftp_A.pcap": "ftp",
        "vpn_sftp_B.pcap": "ftp",
        "vpn_skype_files1a.pcap": "ftp",
        "vpn_skype_files1b.pcap": "ftp",
        # --- streaming ---
        "vpn_vimeo_A.pcap": "streaming",
        "vpn_vimeo_B.pcap": "streaming",
        "vpn_youtube_A.pcap": "streaming",
        "vpn_spotify_A.pcap": "streaming",
        "vpn_netflix_A.pcap": "streaming",
        # --- p2p ---
        "vpn_bittorrent.pcap": "p2p"
    }

# Example usage for your specific case:
def organize_kaggle_pcaps(custom_file_label_map=None):
    """
    Specific function for organizing the Kaggle PCAP files.
    
    Args:
        custom_file_label_map (dict): Optional custom mapping to override defaults
    """
    input_dirs = [
        "/kaggle/input/pcap-data/ISCX-VPN-PCAPS-APP",
        "/kaggle/input/pcap-data/ISCX-VPN-PCAPs-02"
    ]
    
    output_dir = "/kaggle/working/ISCX_VPN_pcap"
    
    final_labels = ["browsing", "chat", "email", "ftp", "p2p", "streaming", "voip"]
    
    results = organize_pcap_files(
        input_directories=input_dirs,
        output_directory=output_dir,
        final_labels=final_labels,
        file_label_map=custom_file_label_map
    )
    
    return results

In [ ]:
import os
import shutil
from collections import defaultdict

def organize_ustf_tfc_dataset(ustf_tfc_path, output_directory):
    """
    Organize USTF-TFC dataset into 20 labeled subdirectories.
    
    Args:
        ustf_tfc_path (str): Path to the USTF-TFC dataset (/kaggle/input/pcap-data/ustf_tfc)
        output_directory (str): Output directory where organized files will be placed
    
    Returns:
        dict: Summary of files organized by category
    """
    
    # Define the 20 target categories
    target_labels = [
        # Benign applications (13 categories)
        "outlook", "worldofwarcraft", "bittorrent", "skype", "gmail", 
        "facetime", "ftp", "mysql", "smb", "weibo",
        # Malware (10 categories) 
        "tinba", "miuref", "zeus", "htbot", "nsis_ay", "neris", 
        "geodo", "cridex", "shifu", "virut"
    ]
    
    # File mapping for USTF-TFC dataset
    file_label_map = {
        # Benign Applications (in /Benign/ directory)
        "Outlook.pcap": "outlook",
        "WorldOfWarcraft.pcap": "worldofwarcraft", 
        "BitTorrent.pcap": "bittorrent",
        "Skype.pcap": "skype",
        "Gmail.pcap": "gmail",
        "Facetime.pcap": "facetime",
        "FTP.pcap": "ftp",
        "MySQL.pcap": "mysql",
        "SMB-1.pcap": "smb",
        "SMB-2.pcap": "smb", 
        "Weibo-1.pcap": "weibo",
        "Weibo-2.pcap": "weibo",
        "Weibo-3.pcap": "weibo",
        "Weibo-4.pcap": "weibo",
        
        # Malware (in /Malware/ directory and subdirectories)
        "Tinba.pcap": "tinba",
        "Miuref.pcap": "miuref", 
        "Zeus.pcap": "zeus",
        "Htbot.pcap": "htbot",
        "Nsis-ay.pcap": "nsis_ay",
        "Neris.pcap": "neris",
        "Geodo.pcap": "geodo", 
        "Cridex.pcap": "cridex",
        "Shifu.pcap": "shifu",
        "Virut.pcap": "virut"
    }
    
    print(f"🚀 Starting USTF-TFC dataset organization")
    print(f"📂 Source: {ustf_tfc_path}")
    print(f"📁 Output: {output_directory}")
    print(f"🎯 Target categories: {len(target_labels)}")
    
    # Create output directory and subdirectories
    os.makedirs(output_directory, exist_ok=True)
    for label in target_labels:
        label_dir = os.path.join(output_directory, label)
        os.makedirs(label_dir, exist_ok=True)
    
    results = defaultdict(list)
    skipped_files = []
    total_processed = 0
    total_copied = 0
    
    # Function to recursively find and process PCAP files
    def process_directory(directory_path, base_path=""):
        nonlocal total_processed, total_copied
        
        for root, dirs, files in os.walk(directory_path):
            for filename in files:
                if filename.lower().endswith(('.pcap', '.pcapng')):
                    total_processed += 1
                    source_path = os.path.join(root, filename)
                    
                    # Look up label from mapping
                    matched_label = file_label_map.get(filename)
                    
                    if matched_label and matched_label in target_labels:
                        # Copy file to appropriate subdirectory
                        dest_dir = os.path.join(output_directory, matched_label)
                        dest_path = os.path.join(dest_dir, filename)
                        
                        try:
                            shutil.copy2(source_path, dest_path)
                            results[matched_label].append(filename)
                            total_copied += 1
                            
                            # Show path for context
                            relative_path = os.path.relpath(source_path, ustf_tfc_path)
                            print(f"✅ {relative_path} → {matched_label}/")
                        except Exception as e:
                            print(f"❌ Error copying {filename}: {e}")
                            skipped_files.append((filename, f"Copy error: {e}"))
                    else:
                        skipped_files.append((filename, "No matching label found"))
                        relative_path = os.path.relpath(source_path, ustf_tfc_path)
                        print(f"⏭️ Skipped: {relative_path} (no mapping)")
    
    # Process the USTF-TFC dataset
    if os.path.exists(ustf_tfc_path):
        print(f"\n📂 Processing USTF-TFC dataset...")
        process_directory(ustf_tfc_path)
    else:
        print(f"❌ USTF-TFC path does not exist: {ustf_tfc_path}")
        return {}
    
    # Print summary
    print(f"\n{'='*70}")
    print(f"📊 USTF-TFC ORGANIZATION SUMMARY")
    print(f"{'='*70}")
    print(f"📈 Total files processed: {total_processed}")
    print(f"✅ Total files copied: {total_copied}")
    print(f"⏭️ Total files skipped: {len(skipped_files)}")
    
    print(f"\n📁 Files by category:")
    for label in target_labels:
        count = len(results[label])
        print(f"  {label}: {count} files")
        if count > 0:
            # Show all files for small counts, first few for larger counts
            examples = results[label][:3] if len(results[label]) > 3 else results[label]
            for example in examples:
                print(f"    📄 {example}")
            if len(results[label]) > 3:
                print(f"    ... and {len(results[label]) - 3} more")
    
    if skipped_files:
        print(f"\n⏭️ Skipped files:")
        for filename, reason in skipped_files:
            print(f"  📄 {filename}: {reason}")
    
    return dict(results)

def preview_ustf_tfc_organization(ustf_tfc_path):
    """
    Preview how USTF-TFC files would be organized without actually copying them.
    
    Args:
        ustf_tfc_path (str): Path to the USTF-TFC dataset
    
    Returns:
        dict: Preview of organization results
    """
    file_label_map = {
        # Benign Applications
        "Outlook.pcap": "outlook",
        "WorldOfWarcraft.pcap": "worldofwarcraft", 
        "BitTorrent.pcap": "bittorrent",
        "Skype.pcap": "skype",
        "Gmail.pcap": "gmail",
        "Facetime.pcap": "facetime",
        "FTP.pcap": "ftp",
        "MySQL.pcap": "mysql",
        "SMB-1.pcap": "smb",
        "SMB-2.pcap": "smb", 
        "Weibo-1.pcap": "weibo",
        "Weibo-2.pcap": "weibo",
        "Weibo-3.pcap": "weibo",
        "Weibo-4.pcap": "weibo",
        
        # Malware
        "Tinba.pcap": "tinba",
        "Miuref.pcap": "miuref", 
        "Zeus.pcap": "zeus",
        "Htbot.pcap": "htbot",
        "Nsis-ay.pcap": "nsis_ay",
        "Neris.pcap": "neris",
        "Geodo.pcap": "geodo", 
        "Cridex.pcap": "cridex",
        "Shifu.pcap": "shifu",
        "Virut.pcap": "virut"
    }
    
    preview_results = defaultdict(list)
    
    if not os.path.exists(ustf_tfc_path):
        print(f"❌ Path does not exist: {ustf_tfc_path}")
        return {}
    
    for root, dirs, files in os.walk(ustf_tfc_path):
        for filename in files:
            if filename.lower().endswith(('.pcap', '.pcapng')):
                matched_label = file_label_map.get(filename)
                
                if matched_label:
                    preview_results[matched_label].append(filename)
                else:
                    preview_results['skipped'].append(filename)
    
    return dict(preview_results)

def organize_ustf_tfc_kaggle():
    """
    Specific function for organizing USTF-TFC dataset in Kaggle environment.
    """
    ustf_tfc_path = "/kaggle/input/pcap-data/ustf_tfc"
    output_dir = "/kaggle/working/organized_ustf_tfc"
    
    results = organize_ustf_tfc_dataset(
        ustf_tfc_path=ustf_tfc_path,
        output_directory=output_dir
    )
    
    return results

def get_ustf_tfc_mapping():
    """
    Return the exact mapping used for USTF-TFC files.
    """
    return {
        # Benign Applications
        "Outlook.pcap": "outlook",
        "WorldOfWarcraft.pcap": "worldofwarcraft", 
        "BitTorrent.pcap": "bittorrent",
        "Skype.pcap": "skype",
        "Gmail.pcap": "gmail",
        "Facetime.pcap": "facetime",
        "FTP.pcap": "ftp",
        "MySQL.pcap": "mysql",
        "SMB-1.pcap": "smb",
        "SMB-2.pcap": "smb", 
        "Weibo-1.pcap": "weibo",
        "Weibo-2.pcap": "weibo",
        "Weibo-3.pcap": "weibo",
        "Weibo-4.pcap": "weibo",
        
        # Malware
        "Tinba.pcap": "tinba",
        "Miuref.pcap": "miuref", 
        "Zeus.pcap": "zeus",
        "Htbot.pcap": "htbot",
        "Nsis-ay.pcap": "nsis_ay",
        "Neris.pcap": "neris",
        "Geodo.pcap": "geodo", 
        "Cridex.pcap": "cridex",
        "Shifu.pcap": "shifu",
        "Virut.pcap": "virut"
    }


## Flow level treatment 

In [ ]:
def get_memory_usage():
    """Get current memory usage in MB"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024 

def create_session_key(packet):
    """
    Create a session key for flow grouping based on 5-tuple (src_ip, src_port, dst_ip, dst_port, protocol).
    Handles bidirectionality by sorting IP/port pairs.
    """
    try:
        if packet.haslayer('IP'):
            ip_layer = packet['IP']
            src_ip = ip_layer.src
            dst_ip = ip_layer.dst
            
            if packet.haslayer('TCP'):
                src_port = packet['TCP'].sport
                dst_port = packet['TCP'].dport
                protocol = 'TCP'
            elif packet.haslayer('UDP'):
                src_port = packet['UDP'].sport
                dst_port = packet['UDP'].dport
                protocol = 'UDP'
            else:
                # For non-TCP/UDP IP packets (e.g., ICMP), use protocol number and 0 for ports
                src_port = dst_port = 0
                protocol = str(ip_layer.proto) # Use IP protocol number
                
            # Create a bidirectional session key by sorting the (IP, Port) tuples
            # This ensures that A->B and B->A traffic for the same flow gets the same key
            if (src_ip, src_port) < (dst_ip, dst_port):
                return f"{src_ip}:{src_port}-{dst_ip}:{dst_port}-{protocol}"
            else:
                return f"{dst_ip}:{dst_port}-{src_ip}:{src_port}-{protocol}"
        else:
            # For non-IP packets (e.g., ARP, Ethernet-only), create a key based on a hash of the packet
            # This is a fallback and might group dissimilar non-IP packets, but ensures all packets are considered.
            return f"non_ip-{hashlib.md5(bytes(packet)).hexdigest()[:8]}"
    except Exception as e:
        # Fallback for any parsing errors, using a hash of the raw packet bytes for uniqueness
        print(f"Error creating session key: {e}. Falling back to packet hash.")
        return f"error-{hashlib.md5(bytes(packet)).hexdigest()[:8]}"

def extract_flows_streaming(pcap_file, min_packets=5, max_flows_per_batch=100, 
                            memory_limit_mb=1000, max_total_flows=None, timeout_seconds=300):
    """
    Extract flows from large PCAP files using a streaming approach with smart flow detection.
    Efficiently handles flows by using timeout-based cleanup and memory limits.

    Args:
        pcap_file (str): Path to the input PCAP file.
        min_packets (int): Minimum number of packets a flow must have to be considered valid.
        max_flows_per_batch (int): Maximum number of completed flows to accumulate before a batch processing/cleanup.
        memory_limit_mb (int): Memory usage threshold in MB to trigger a cleanup/batch processing.
        max_total_flows (int, optional): Maximum total flows to extract from this PCAP file.
        timeout_seconds (int): Time in seconds after which a flow is considered complete if no new packets are seen.

    Returns:
        list: A list of lists, where each inner list contains packets belonging to a single valid flow.
    """
    print(f"Processing large PCAP file: {pcap_file}")
    print(f"Memory limit: {memory_limit_mb}MB, Max flows per batch: {max_flows_per_batch}")
    print(f"Min packets per flow: {min_packets}, Flow timeout: {timeout_seconds}s")
    
    try:
        sessions = defaultdict(list)  # Stores active flows (session_key -> list of packets)
        session_last_seen = {}       # Tracks the timestamp of the last packet for each session
        valid_flows = []             # Stores completed flows that meet min_packets criteria
        
        packet_count = 0
        batch_count = 0
        total_extracted_flows = 0
        discarded_short_flows = 0
        
        with PcapReader(pcap_file) as pcap_reader:
            start_memory = get_memory_usage()
            last_cleanup_time = time.time()
            
            for packet in pcap_reader:
                packet_count += 1
                current_real_time = time.time() # Wall clock time for periodic checks
                packet_timestamp = packet.time  # Scapy's packet timestamp for flow timeout
                
                session_key = create_session_key(packet)
                sessions[session_key].append(packet)
                session_last_seen[session_key] = packet_timestamp
                
                # Periodic cleanup of old/incomplete sessions
                # This helps manage memory and finalize flows that have ended
                if (packet_count % 5000 == 0 or 
                    current_real_time - last_cleanup_time > 30): # Cleanup every 5000 packets or 30 seconds
                    
                    sessions_to_cleanup = []
                    added_this_cleanup_cycle = 0
                    discarded_this_cleanup_cycle = 0
                    
                    # Iterate through active sessions to find those that have timed out
                    for session_key, last_timestamp in list(session_last_seen.items()): # Use list to allow deletion during iteration
                        time_since_last_packet = packet_timestamp - last_timestamp
                        
                        if time_since_last_packet > timeout_seconds:
                            session_packet_count = len(sessions[session_key])
                            
                            if session_packet_count >= min_packets:
                                # This flow is complete and meets minimum packet requirement
                                session_packets = sessions[session_key]
                                session_packets.sort(key=lambda p: p.time) # Ensure packets are sorted by time
                                valid_flows.append(session_packets)
                                total_extracted_flows += 1
                                added_this_cleanup_cycle += 1
                            else:
                                # Discard flow with insufficient packets
                                discarded_short_flows += 1
                                discarded_this_cleanup_cycle += 1
                                
                            sessions_to_cleanup.append(session_key)
                    
                    # Remove cleaned up sessions from active tracking dictionaries
                    for key in sessions_to_cleanup:
                        del sessions[key]
                        del session_last_seen[key]
                    
                    if sessions_to_cleanup:
                        print(f"Cleanup: Added {added_this_cleanup_cycle} flows, "
                              f"discarded {discarded_this_cleanup_cycle} short flows. Active sessions remaining: {len(sessions)}")
                    
                    last_cleanup_time = current_real_time
                
                # Check memory usage and batch processing periodically
                if packet_count % 1000 == 0:
                    current_memory = get_memory_usage()
                    memory_used = current_memory - start_memory
                    
                    # Count sessions that currently have enough packets to be considered valid
                    potential_valid_sessions = len([s for s in sessions.values() if len(s) >= min_packets])
                    
                    print(f"Processed {packet_count} packets, Memory: {memory_used:.1f}MB, "
                          f"Active sessions: {len(sessions)}, Potential flows: {potential_valid_sessions}, "
                          f"Completed flows (total): {total_extracted_flows}, Discarded (total): {discarded_short_flows}")
                    
                    # If memory usage is high or we have accumulated enough completed flows,
                    # clear the `valid_flows` list to free memory and reset tracking.
                    if (memory_used > memory_limit_mb or 
                        len(valid_flows) >= max_flows_per_batch or
                        potential_valid_sessions >= max_flows_per_batch):
                        
                        batch_count += 1
                        print(f"Batch {batch_count}: Processing {len(valid_flows)} completed flows (accumulated)")
                        
                        # If a max_total_flows limit is set, check if we've reached it.
                        # This break ensures we don't process more than needed from this file.
                        if max_total_flows and total_extracted_flows >= max_total_flows:
                            print(f"Reached maximum total flows limit for this file: {max_total_flows}")
                            # Ensure valid_flows doesn't exceed the limit if we break early
                            valid_flows = valid_flows[:max_total_flows - (total_extracted_flows - len(valid_flows))]
                            break # Exit the packet reading loop
                        
                        # Clear accumulated valid flows from memory, as they will be written to disk soon
                        if len(valid_flows) >= max_flows_per_batch: # Only clear if we actually have a batch to process
                            valid_flows.clear()
                        
                        # Force garbage collection to ensure memory is freed
                        gc.collect()
                        
                        # Reset memory tracking for the next batch
                        start_memory = get_memory_usage()
            
            # Final processing of any remaining active sessions after all packets are read
            print("Processing remaining sessions after end of file...")
            final_flows_added = 0
            final_flows_discarded = 0
            
            for session_key, session_packets in sessions.items():
                if len(session_packets) >= min_packets:
                    # If max_total_flows is set, check before adding more flows
                    if max_total_flows and total_extracted_flows >= max_total_flows:
                        print(f"Reached maximum total flows limit ({max_total_flows}) during final processing.")
                        break # Stop adding flows
                    session_packets.sort(key=lambda p: p.time)
                    valid_flows.append(session_packets)
                    total_extracted_flows += 1
                    final_flows_added += 1
                else:
                    final_flows_discarded += 1
            
            print(f"Final processing: Added {final_flows_added} flows, "
                  f"discarded {final_flows_discarded} short flows.")
            print(f"Total flows extracted from {pcap_file}: {total_extracted_flows}")
            print(f"Total short flows discarded from {pcap_file}: {discarded_short_flows + final_flows_discarded}")
            
            # Clear all session data from memory
            sessions.clear()
            session_last_seen.clear()
            gc.collect()
            
            # Return flows, respecting the max_total_flows limit if set
            return valid_flows[:max_total_flows] if max_total_flows else valid_flows
            
    except Exception as e:
        print(f"Error processing {pcap_file}: {e}")
        return []

def write_flows_in_chunks(flows, output_dir, base_filename, chunk_size=50):
    """
    Write flows to files in chunks to avoid memory issues when dealing with many flows.

    Args:
        flows (list): A list of flow packets (each flow is a list of Scapy packets).
        output_dir (str): Directory where flow PCAP files will be saved.
        base_filename (str): Base name for the output flow files (derived from original PCAP).
        chunk_size (int): Number of flows to process and write in each chunk.

    Returns:
        int: The total number of flows successfully written to disk.
    """
    os.makedirs(output_dir, exist_ok=True)
    written_count = 0
    
    print(f"Starting to write {len(flows)} flows to {output_dir} in chunks of {chunk_size}...")
    
    # Iterate through flows in specified chunk sizes
    for i in tqdm.tqdm(range(0, len(flows), chunk_size), desc="Writing flows"):
        chunk_flows = flows[i:i + chunk_size]
        
        # Write each flow within the current chunk
        for j, flow_packets in enumerate(chunk_flows):
            flow_index = i + j
            # Create a unique hash for the filename to prevent collisions, especially if multiple PCAPs
            # contribute to the same label and flow index.
            session_hash = hashlib.md5(f"{base_filename}_{flow_index}_{time.time_ns()}_{random.randint(0, 10000)}".encode()).hexdigest()[:8]
            output_file = os.path.join(output_dir, f"{base_filename}_flow_{flow_index:06d}_{session_hash}.pcap")
            
            try:
                wrpcap(output_file, flow_packets)
                written_count += 1
            except Exception as e:
                print(f"Error writing flow {flow_index} to {output_file}: {e}")
        
        # Explicitly delete the chunk and force garbage collection to free memory
        del chunk_flows
        gc.collect()
        
        print(f"Written {written_count} flows so far for {base_filename}...")
    
    print(f"Finished writing flows. Total written: {written_count}")
    return written_count

def process_large_pcap_efficiently(pcap_file, output_dir, label_name, 
                                     max_flows_needed=1000, min_packets=5,
                                     memory_limit_mb=800, max_flows_per_batch=100,
                                     flow_timeout=300):
    """
    Process a single large PCAP file, extract flows, and write them to disk.
    This function acts as an orchestrator for large files.

    Args:
        pcap_file (str): Path to the input PCAP file.
        output_dir (str): Base output directory.
        label_name (str): Label associated with the PCAP file (e.g., 'malware', 'benign').
        max_flows_needed (int): Maximum number of flows to extract from this specific PCAP file.
        min_packets (int): Minimum number of packets a flow must have.
        memory_limit_mb (int): Memory threshold for streaming extraction.
        max_flows_per_batch (int): Batch size for flow accumulation during streaming.
        flow_timeout (int): Timeout for flow completion in seconds.

    Returns:
        int: The number of flows successfully written for this PCAP file.
    """
    print(f"\n{'='*60}")
    print(f"Processing large PCAP: {os.path.basename(pcap_file)}")
    print(f"Target flows from this file: {max_flows_needed}")
    print(f"{'='*60}")
    
    start_time = time.time()
    
    # Extract flows using the streaming approach
    flows = extract_flows_streaming(
        pcap_file=pcap_file,
        min_packets=min_packets,
        max_flows_per_batch=max_flows_per_batch,
        memory_limit_mb=memory_limit_mb,
        max_total_flows=max_flows_needed, # Pass the limit to the extractor
        timeout_seconds=flow_timeout
    )
    
    if not flows:
        print(f"No valid flows found in {pcap_file} after extraction.")
        return 0
    
    print(f"Extracted {len(flows)} flows from {pcap_file} in {time.time() - start_time:.1f} seconds.")
    
    # Shuffle the extracted flows to get a random sample if more were extracted than needed
    if len(flows) > max_flows_needed:
        random.shuffle(flows)
        flows_to_write = flows[:max_flows_needed]
        print(f"Limiting to {len(flows_to_write)} flows for writing (due to max_flows_needed).")
    else:
        flows_to_write = flows
    
    # Determine the output directory for this label
    base_filename = os.path.splitext(os.path.basename(pcap_file))[0]
    label_output_dir = os.path.join(output_dir, label_name)
    
    # Write the selected flows to disk in chunks
    written_count = write_flows_in_chunks(
        flows=flows_to_write,
        output_dir=label_output_dir,
        base_filename=base_filename,
        chunk_size=50  # Write 50 flows at a time to manage memory
    )
    
    # Clear all flow data from memory after writing
    flows.clear()
    flows_to_write.clear()
    gc.collect()
    
    print(f"Successfully wrote {written_count} flows for label '{label_name}' from {os.path.basename(pcap_file)}.")
    return written_count

def process_regular_pcap_file(pcap_file, output_dir, label_name, max_flows_needed, min_packets=5):
    """
    Process a regular-sized PCAP file by loading all packets into memory,
    extracting flows, and writing them to disk.
    This is suitable for smaller PCAP files where full loading is feasible.

    Args:
        pcap_file (str): Path to the input PCAP file.
        output_dir (str): Base output directory.
        label_name (str): Label associated with the PCAP file.
        max_flows_needed (int): Maximum number of flows to extract from this specific PCAP file.
        min_packets (int): Minimum number of packets a flow must have.

    Returns:
        int: The number of flows successfully written for this PCAP file.
    """
    print(f"\n📄 Processing regular file: {os.path.basename(pcap_file)}")
    print(f"Target flows from this file: {max_flows_needed}")
    
    try:
        # Load all packets into memory (suitable for smaller files)
        packets = rdpcap(pcap_file)
        if len(packets) == 0:
            print(f"No packets found in {pcap_file}.")
            return 0
            
        # Group packets by session key
        sessions = defaultdict(list)
        for packet in packets:
            session_key = create_session_key(packet)
            sessions[session_key].append(packet)
        
        # Filter for valid flows (meeting min_packets criteria)
        valid_flows = []
        for session_key, session_packets in sessions.items():
            if len(session_packets) >= min_packets:
                session_packets.sort(key=lambda p: p.time) # Sort packets within the flow by timestamp
                valid_flows.append(session_packets)
        
        # Limit the number of flows and shuffle for random sampling if more are available
        if len(valid_flows) > max_flows_needed:
            print(f"Found {len(valid_flows)} flows, sampling down to {max_flows_needed}.")
            valid_flows = random.sample(valid_flows, max_flows_needed)
        else:
            print(f"Found {len(valid_flows)} flows, writing all of them.")
            
        # Determine the output directory for this label
        base_filename = os.path.splitext(os.path.basename(pcap_file))[0]
        label_output_dir = os.path.join(output_dir, label_name)
        os.makedirs(label_output_dir, exist_ok=True)
        
        written_count = 0
        # Write each valid flow to a separate PCAP file
        for i, flow_packets in enumerate(valid_flows):
            # Create a unique hash for the filename
            session_hash = hashlib.md5(f"{pcap_file}_{i}_{time.time_ns()}_{random.randint(0, 10000)}".encode()).hexdigest()[:8]
            output_file = os.path.join(label_output_dir, f"{base_filename}_flow_{i:04d}_{session_hash}.pcap")
            
            try:
                wrpcap(output_file, flow_packets)
                written_count += 1
            except Exception as e:
                print(f"Error writing flow {i} to {output_file}: {e}")
        
        # Clear memory after processing
        del packets
        sessions.clear()
        valid_flows.clear()
        gc.collect()

        return written_count
        
    except Exception as e:
        print(f"Error processing {pcap_file}: {e}")
        return 0

def process_directory_with_large_files(input_dir, output_dir, samples_per_label=2000, # Changed default to 2000 as per your requirement
                                       min_packets=3, # Changed default to 3 as per your requirement
                                       memory_limit_mb=800, 
                                       large_file_threshold_mb=500):
    """
    Process a directory containing PCAP files, organizing them by label (subdirectories).
    It intelligently chooses between streaming and full-load processing based on file size.

    Args:
        input_dir (str): The root directory containing label subdirectories with PCAP files.
        output_dir (str): The base directory where processed flows will be saved.
        samples_per_label (int): The maximum number of flow files to generate for each label.
        min_packets (int): The minimum number of packets a flow must contain.
        memory_limit_mb (int): Memory threshold for activating streaming mode for large files.
        large_file_threshold_mb (int): File size in MB above which streaming mode is used.

    Returns:
        dict: A dictionary showing the total number of flows extracted for each label.
    """
    total_processed = {}
    
    # Iterate through each label directory within the input directory
    for label_name in os.listdir(input_dir):
        label_path = os.path.join(input_dir, label_name)
        if not os.path.isdir(label_path):
            continue # Skip if it's not a directory
            
        print(f"\n🏷️ Processing label: {label_name}")
        
        # Collect all PCAP files for the current label
        pcap_files = []
        for root, dirs, files in os.walk(label_path):
            for file in files:
                if file.lower().endswith(('.pcap', '.pcapng')):
                    file_path = os.path.join(root, file)
                    file_size_mb = os.path.getsize(file_path) / (1024 * 1024)
                    pcap_files.append((file_path, file_size_mb))
        
        if not pcap_files:
            print(f"No PCAP files found for {label_name}. Skipping.")
            continue
        
        # Sort files by size (smaller files first) to efficiently meet samples_per_label
        pcap_files.sort(key=lambda x: x[1])
        
        print(f"Found {len(pcap_files)} PCAP files for {label_name}:")
        for file_path, size_mb in pcap_files:
            print(f"  📁 {os.path.basename(file_path)}: {size_mb:.1f}MB")
        
        # Process files until the required number of samples per label is met
        extracted_count = 0
        for file_path, file_size_mb in pcap_files:
            if extracted_count >= samples_per_label:
                print(f"Reached target {samples_per_label} flows for {label_name}. Skipping remaining files.")
                break # Stop processing files for this label if target is met
            
            remaining_needed = samples_per_label - extracted_count
            
            if file_size_mb > large_file_threshold_mb:
                print(f"⚠️ Large file detected: {os.path.basename(file_path)} ({file_size_mb:.1f}MB). Using streaming mode.")
                # Use streaming approach for very large files
                flows_extracted = process_large_pcap_efficiently(
                    pcap_file=file_path,
                    output_dir=output_dir,
                    label_name=label_name,
                    max_flows_needed=remaining_needed, # Only extract up to the remaining needed flows
                    min_packets=min_packets,
                    memory_limit_mb=memory_limit_mb
                )
            else:
                print(f"📄 Processing regular file: {os.path.basename(file_path)} ({file_size_mb:.1f}MB). Using standard mode.")
                # Use regular approach for smaller files
                flows_extracted = process_regular_pcap_file(
                    pcap_file=file_path,
                    output_dir=output_dir,
                    label_name=label_name,
                    max_flows_needed=remaining_needed, # Only extract up to the remaining needed flows
                    min_packets=min_packets
                )
            
            extracted_count += flows_extracted
            print(f"✅ Progress for {label_name}: {extracted_count}/{samples_per_label} flows extracted so far.")
            
            # Force garbage collection between processing different files
            gc.collect()
            
        total_processed[label_name] = extracted_count
        print(f"🎯 Completed {label_name}: {extracted_count} flows extracted.")
    
    return total_processed

def verify_flows_with_min_packets(pcap_file, min_packets=3):
    """
    Verifies and counts the number of flows in a PCAP file that meet a minimum packet count.
    This function is designed to quickly assess the potential number of extractable flows
    without writing them to disk.

    Args:
        pcap_file (str): Path to the input PCAP file.
        min_packets (int): The minimum number of packets a flow must have to be counted.

    Returns:
        int: The total count of flows that satisfy the min_packets criteria.
    """
    print(f"\n🔬 Verifying flows in {os.path.basename(pcap_file)} with at least {min_packets} packets...")
    
    sessions = defaultdict(list)
    flow_count = 0
    
    try:
        with PcapReader(pcap_file) as pcap_reader:
            for packet in pcap_reader:
                session_key = create_session_key(packet)
                sessions[session_key].append(packet) # Just append, no need for full packet objects if only counting
        
        for session_key, session_packets in sessions.items():
            if len(session_packets) >= min_packets:
                flow_count += 1
        
        print(f"✅ Verification complete: Found {flow_count} flows in {os.path.basename(pcap_file)} meeting the {min_packets}-packet criterion.")
        return flow_count
    except Exception as e:
        print(f"Error verifying {pcap_file}: {e}")
        return 0

# Kaggle-optimized usage example
def process_pcaps_for_kaggle(input_path, output_path, samples_per_label=2000): # Updated default
    """
    Main function optimized for a Kaggle-like environment.
    This function serves as the entry point for running the PCAP processing.

    Args:
        input_path (str): Path to the directory containing PCAP files, organized by label.
        output_path (str): Path to the directory where the processed flow files will be saved.
        samples_per_label (int): The target number of flow files for each label.
    """
    print("🚀 Starting PCAP processing optimized for Kaggle")
    print(f"📊 Memory available: {psutil.virtual_memory().available / (1024**3):.1f}GB")
    print(f"🎯 Target samples per label: {samples_per_label}")
    
    # Use conservative memory settings for Kaggle environments, which often have limited RAM
    results = process_directory_with_large_files(
        input_dir=input_path,
        output_dir=output_path,
        samples_per_label=samples_per_label,
        min_packets=3,  # Set to 3 as per your requirement
        memory_limit_mb=800,  # Conservative memory limit for Kaggle
        large_file_threshold_mb=300 # Lower threshold to use streaming earlier for Kaggle
    )
    
    print("\n📈 Final Results:")
    total_flows = 0
    for label, count in results.items():
        print(f"  {label}: {count} flows")
        total_flows += count
    
    print(f"🎉 Total flows extracted across all labels: {total_flows}")
    return results


## Packet level 

In [ ]:
def get_memory_usage():
    """Get current memory usage in MB"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024

def cut(obj, sec):
    """Cut object into sections"""
    if not obj or len(obj) == 0:
        return []
        
    result = [obj[i:i+sec] for i in range(0,len(obj),sec)]
    
    if not result or len(result) == 0:
        return []
        
    remanent_count = len(result[0])%4
    if remanent_count == 0:
        pass
    else:
        result = [obj[i:i+sec+remanent_count] for i in range(0,len(obj),sec+remanent_count)]
    
    return result

def bigram_generation(packet_datagram, packet_len=64, flag=True):
    """Generate bigrams from packet data"""
    if not packet_datagram or len(packet_datagram) == 0:
        return ''
    
    result = ''
    generated_datagram = cut(packet_datagram, 1)
    
    if not generated_datagram or len(generated_datagram) == 0:
        return ''
    
    token_count = 0
    for sub_string_index in range(len(generated_datagram)):
        if sub_string_index != (len(generated_datagram) - 1):
            token_count += 1
            if token_count > packet_len:
                break
            else:
                merge_word_bigram = generated_datagram[sub_string_index] + generated_datagram[sub_string_index + 1]
        else:
            break
        result += merge_word_bigram
        result += ' '
    
    return result

def size_format(size):
    """Format file size in KB"""
    return '%.3f' % float(size/1000)

def is_valid_packet(packet, min_size_bytes=80):
    """
    Check if a packet is valid for processing.
    
    Args:
        packet: Scapy packet object
        min_size_bytes: Minimum packet size in bytes
    
    Returns:
        bool: True if packet is valid, False otherwise
    """
    try:
        # Check packet size
        if len(packet) < min_size_bytes:
            return False
            
        # Check if it's an IP packet (IPv4 or IPv6)
        if not (packet.haslayer('IP') or packet.haslayer('IPv6')):
            return False
            
        # Filter out common non-data protocols
        if (packet.haslayer('ARP') or packet.haslayer('DNS') or 
            packet.haslayer('DHCP') or packet.haslayer('ICMPv6') or
            packet.haslayer('ICMP')):
            return False
            
        return True
    except:
        return False

def extract_packet_feature(packet, payload_len=128):
    """
    Extract features from a single packet.
    
    Args:
        packet: Scapy packet object
        payload_len: Maximum payload length to process
    
    Returns:
        str: Processed packet data as bigram string
    """
    try:
        packet_data = packet.copy()
        data = binascii.hexlify(bytes(packet_data))
        packet_string = data.decode()
        
        # Skip Ethernet header (first 76 characters = 38 bytes)
        new_packet_string = packet_string[76:] if len(packet_string) > 76 else packet_string
        
        # Generate bigrams
        result = bigram_generation(new_packet_string, packet_len=payload_len, flag=True)
        return result
    except Exception as e:
        print(f"Error extracting packet feature: {e}")
        return ''

def extract_packets_streaming(pcap_file, max_packets=5000, memory_limit_mb=1000, 
                            min_packet_size=80, payload_len=128):
    """
    Extract packets from large PCAP files using streaming approach.
    
    Args:
        pcap_file (str): Path to the input PCAP file
        max_packets (int): Maximum number of packets to extract
        memory_limit_mb (int): Memory usage threshold in MB
        min_packet_size (int): Minimum packet size in bytes
        payload_len (int): Payload length for feature extraction
    
    Returns:
        list: List of valid packet objects (not features)
    """
    print(f"Processing large PCAP file: {pcap_file}")
    print(f"Memory limit: {memory_limit_mb}MB, Max packets: {max_packets}")
    print(f"Min packet size: {min_packet_size} bytes")
    
    try:
        valid_packets = []  # Store actual packet objects instead of features
        packet_count = 0
        valid_packet_count = 0
        batch_size = 1000
        
        with PcapReader(pcap_file) as pcap_reader:
            start_memory = get_memory_usage()
            
            for packet in pcap_reader:
                packet_count += 1
                
                # Check if packet is valid for processing
                if not is_valid_packet(packet, min_packet_size):
                    continue
                
                # Store the actual packet object instead of extracting features
                valid_packets.append(packet.copy())  # Use copy() to ensure packet is preserved
                valid_packet_count += 1
                
                # Check if we've reached the maximum number of packets
                if valid_packet_count >= max_packets:
                    print(f"Reached maximum packet limit: {max_packets}")
                    break
                
                # Memory and progress check every batch
                if packet_count % batch_size == 0:
                    current_memory = get_memory_usage()
                    memory_used = current_memory - start_memory
                    
                    print(f"Processed {packet_count} packets, Valid: {valid_packet_count}, "
                          f"Memory: {memory_used:.1f}MB")
                    
                    # If memory usage is too high, break
                    if memory_used > memory_limit_mb:
                        print(f"Memory limit reached ({memory_limit_mb}MB), stopping extraction")
                        break
                    
                    # Force garbage collection periodically
                    if packet_count % (batch_size * 5) == 0:
                        gc.collect()
            
            print(f"Extraction complete: {valid_packet_count} valid packets from {packet_count} total packets")
            return valid_packets
            
    except Exception as e:
        print(f"Error processing {pcap_file}: {e}")
        return []

def write_packets_in_chunks(valid_packets, output_dir, base_filename, chunk_size=100):
    """
    Write packet objects to PCAP files in chunks.
    
    Args:
        valid_packets (list): List of Scapy packet objects
        output_dir (str): Directory to write PCAP files
        base_filename (str): Base name for output files
        chunk_size (int): Number of packets to process in each chunk
    
    Returns:
        int: Number of packets successfully written
    """
    os.makedirs(output_dir, exist_ok=True)
    written_count = 0
    
    print(f"Writing {len(valid_packets)} packets to {output_dir} in chunks of {chunk_size}...")
    
    # Write packets in chunks to manage memory
    for i in tqdm.tqdm(range(0, len(valid_packets), chunk_size), desc="Writing packets"):
        chunk_packets = valid_packets[i:i + chunk_size]
        
        # Write each packet to a separate PCAP file
        for j, packet in enumerate(chunk_packets):
            packet_index = i + j
            # Create unique filename
            packet_hash = hashlib.md5(f"{base_filename}_{packet_index}_{time.time_ns()}_{random.randint(0, 10000)}".encode()).hexdigest()[:8]
            output_file = os.path.join(output_dir, f"{base_filename}_packet_{packet_index:06d}_{packet_hash}.pcap")
            
            try:
                # Write single packet to PCAP file
                wrpcap(output_file, [packet])
                written_count += 1
            except Exception as e:
                print(f"Error writing packet {packet_index} to {output_file}: {e}")
        
        # Clean up chunk from memory
        del chunk_packets
        gc.collect()
        
        if i % (chunk_size * 10) == 0:  # Progress update every 10 chunks
            print(f"Written {written_count} packets so far for {base_filename}...")
    
    print(f"Finished writing packets. Total written: {written_count}")
    return written_count

def process_large_pcap_packet_level(pcap_file, output_dir, label_name, 
                                   max_packets_needed=5000, min_packet_size=80,
                                   memory_limit_mb=800, payload_len=128):
    """
    Process a single large PCAP file for packet-level analysis.
    
    Args:
        pcap_file (str): Path to input PCAP file
        output_dir (str): Base output directory
        label_name (str): Label for the PCAP file
        max_packets_needed (int): Maximum packets to extract
        min_packet_size (int): Minimum packet size in bytes
        memory_limit_mb (int): Memory threshold
        payload_len (int): Payload length for processing (unused now but kept for compatibility)
    
    Returns:
        int: Number of packets successfully processed
    """
    print(f"\n{'='*60}")
    print(f"Processing large PCAP for packets: {os.path.basename(pcap_file)}")
    print(f"Target packets from this file: {max_packets_needed}")
    print(f"{'='*60}")
    
    start_time = time.time()
    
    # Extract valid packets using streaming approach
    valid_packets = extract_packets_streaming(
        pcap_file=pcap_file,
        max_packets=max_packets_needed,
        memory_limit_mb=memory_limit_mb,
        min_packet_size=min_packet_size,
        payload_len=payload_len
    )
    
    if not valid_packets:
        print(f"No valid packets found in {pcap_file}")
        return 0
    
    print(f"Extracted {len(valid_packets)} valid packets from {pcap_file} in {time.time() - start_time:.1f} seconds")
    
    # Shuffle and limit if needed
    if len(valid_packets) > max_packets_needed:
        random.shuffle(valid_packets)
        packets_to_write = valid_packets[:max_packets_needed]
        print(f"Limiting to {len(packets_to_write)} packets for writing")
    else:
        packets_to_write = valid_packets
    
    # Determine output directory
    base_filename = os.path.splitext(os.path.basename(pcap_file))[0]
    label_output_dir = os.path.join(output_dir, label_name)
    
    # Write packet objects to PCAP files
    written_count = write_packets_in_chunks(
        valid_packets=packets_to_write,
        output_dir=label_output_dir,
        base_filename=base_filename,
        chunk_size=100
    )
    
    # Clean up memory
    valid_packets.clear()
    packets_to_write.clear()
    gc.collect()
    
    print(f"Successfully wrote {written_count} packets for label '{label_name}' from {os.path.basename(pcap_file)}")
    return written_count

def process_regular_pcap_packet_level(pcap_file, output_dir, label_name, 
                                     max_packets_needed, min_packet_size=80, payload_len=128):
    """
    Process a regular-sized PCAP file for packet-level analysis.
    
    Args:
        pcap_file (str): Path to input PCAP file
        output_dir (str): Base output directory
        label_name (str): Label for the PCAP file
        max_packets_needed (int): Maximum packets to extract
        min_packet_size (int): Minimum packet size in bytes
        payload_len (int): Payload length for processing (unused now but kept for compatibility)
    
    Returns:
        int: Number of packets successfully processed
    """
    print(f"\n📄 Processing regular file for packets: {os.path.basename(pcap_file)}")
    print(f"Target packets from this file: {max_packets_needed}")
    
    try:
        # Load all packets
        packets = rdpcap(pcap_file)
        if len(packets) == 0:
            print(f"No packets found in {pcap_file}")
            return 0
        
        # Filter valid packets
        valid_packets = []
        for packet in packets:
            if is_valid_packet(packet, min_packet_size):
                valid_packets.append(packet.copy())  # Store actual packet objects
        
        print(f"Found {len(valid_packets)} valid packets out of {len(packets)} total")
        
        # Sample if needed
        if len(valid_packets) > max_packets_needed:
            print(f"Sampling {max_packets_needed} packets from {len(valid_packets)} available")
            valid_packets = random.sample(valid_packets, max_packets_needed)
        
        # Write packets to PCAP files
        base_filename = os.path.splitext(os.path.basename(pcap_file))[0]
        label_output_dir = os.path.join(output_dir, label_name)
        os.makedirs(label_output_dir, exist_ok=True)
        
        written_count = 0
        for i, packet in enumerate(valid_packets):
            packet_hash = hashlib.md5(f"{pcap_file}_{i}_{time.time_ns()}_{random.randint(0, 10000)}".encode()).hexdigest()[:8]
            output_file = os.path.join(label_output_dir, f"{base_filename}_packet_{i:04d}_{packet_hash}.pcap")
            
            try:
                # Write single packet to PCAP file
                wrpcap(output_file, [packet])
                written_count += 1
            except Exception as e:
                print(f"Error writing packet {i}: {e}")
        
        # Clean up memory
        del packets
        valid_packets.clear()
        gc.collect()
        
        return written_count
        
    except Exception as e:
        print(f"Error processing {pcap_file}: {e}")
        return 0

def process_directory_packet_level(input_dir, output_dir, samples_per_label=5000,
                                  min_packet_size=80, payload_len=128,
                                  memory_limit_mb=800, large_file_threshold_mb=500):
    """
    Process directory for packet-level analysis with large file handling.
    
    Args:
        input_dir (str): Input directory with label subdirectories
        output_dir (str): Output directory for processed packets
        samples_per_label (int): Target number of packets per label
        min_packet_size (int): Minimum packet size in bytes
        payload_len (int): Payload length for processing
        memory_limit_mb (int): Memory threshold for streaming
        large_file_threshold_mb (int): File size threshold for streaming mode
    
    Returns:
        dict: Results showing packets extracted per label
    """
    total_processed = {}
    
    for label_name in os.listdir(input_dir):
        label_path = os.path.join(input_dir, label_name)
        if not os.path.isdir(label_path):
            continue
        
        print(f"\n🏷️ Processing label for packets: {label_name}")
        
        # Collect PCAP files
        pcap_files = []
        for root, dirs, files in os.walk(label_path):
            for file in files:
                if file.lower().endswith(('.pcap', '.pcapng')):
                    file_path = os.path.join(root, file)
                    file_size_mb = os.path.getsize(file_path) / (1024 * 1024)
                    pcap_files.append((file_path, file_size_mb))
        
        if not pcap_files:
            print(f"No PCAP files found for {label_name}")
            continue
        
        # Sort by size (smaller first)
        pcap_files.sort(key=lambda x: x[1])
        
        print(f"Found {len(pcap_files)} PCAP files for {label_name}:")
        for file_path, size_mb in pcap_files:
            print(f"  📁 {os.path.basename(file_path)}: {size_mb:.1f}MB")
        
        # Process files until target is met
        extracted_count = 0
        for file_path, file_size_mb in pcap_files:
            if extracted_count >= samples_per_label:
                print(f"Reached target {samples_per_label} packets for {label_name}")
                break
            
            remaining_needed = samples_per_label - extracted_count
            
            if file_size_mb > large_file_threshold_mb:
                print(f"⚠️ Large file detected: {os.path.basename(file_path)} ({file_size_mb:.1f}MB). Using streaming mode.")
                packets_extracted = process_large_pcap_packet_level(
                    pcap_file=file_path,
                    output_dir=output_dir,
                    label_name=label_name,
                    max_packets_needed=remaining_needed,
                    min_packet_size=min_packet_size,
                    memory_limit_mb=memory_limit_mb,
                    payload_len=payload_len
                )
            else:
                print(f"📄 Processing regular file: {os.path.basename(file_path)} ({file_size_mb:.1f}MB)")
                packets_extracted = process_regular_pcap_packet_level(
                    pcap_file=file_path,
                    output_dir=output_dir,
                    label_name=label_name,
                    max_packets_needed=remaining_needed,
                    min_packet_size=min_packet_size,
                    payload_len=payload_len
                )
            
            extracted_count += packets_extracted
            print(f"✅ Progress for {label_name}: {extracted_count}/{samples_per_label} packets")
            
            gc.collect()
        
        total_processed[label_name] = extracted_count
        print(f"🎯 Completed {label_name}: {extracted_count} packets extracted")
    
    return total_processed

def process_pcaps_packet_level_kaggle(input_path, output_path, samples_per_label=5000):
    """
    Main function for Kaggle packet-level processing.
    
    Args:
        input_path (str): Input directory path
        output_path (str): Output directory path
        samples_per_label (int): Target packets per label
    
    Returns:
        dict: Processing results
    """
    print("🚀 Starting packet-level PCAP processing optimized for Kaggle")
    print(f"📊 Memory available: {psutil.virtual_memory().available / (1024**3):.1f}GB")
    print(f"🎯 Target packets per label: {samples_per_label}")
    
    results = process_directory_packet_level(
        input_dir=input_path,
        output_dir=output_path,
        samples_per_label=samples_per_label,
        min_packet_size=80,
        payload_len=128,
        memory_limit_mb=800,
        large_file_threshold_mb=300
    )
    
    print("\n📈 Final Results:")
    total_packets = 0
    for label, count in results.items():
        print(f"  {label}: {count} packets")
        total_packets += count
    
    print(f"🎉 Total packets extracted across all labels: {total_packets}")
    return results


## dataset.json creation

In [ ]:
def filter_packets(packets):
    """
    Filter out ARP, DHCP and other irrelevant packets according to research rules
    """
    filtered_packets = []
    
    for packet in packets:
        # Skip if packet is too small (< 80 bytes as mentioned in rules)
        if len(packet) <= 80:
            continue
            
        # Skip ARP packets (as per rules)
        if packet.haslayer(ARP):
            continue
            
        # Skip DHCP packets (as per rules)
        if packet.haslayer(DHCP):
            continue
            
        # Skip other irrelevant protocols
        if packet.haslayer(DNS):
            continue
        if packet.haslayer(ICMP):
            continue
            
        # Skip ICMPv6 packets (check by IPv6 layer and next header)
        if packet.haslayer(IPv6):
            # ICMPv6 has protocol number 58 in IPv6
            if packet[IPv6].nh == 58:  # Next Header = ICMPv6
                continue
            
        # Only keep TCP and UDP packets with IP layer
        if packet.haslayer(IP) and (packet.haslayer(TCP) or packet.haslayer(UDP)):
            filtered_packets.append(packet)
        # Also accept IPv6 TCP/UDP packets
        elif packet.haslayer(IPv6) and (packet.haslayer(TCP) or packet.haslayer(UDP)):
            filtered_packets.append(packet)
    
    return filtered_packets

def extract_payload_only(packet):
    """
    Extract only payload data, removing all headers as per research rules:
    - Ethernet header
    - IP header  
    - TCP/UDP header and ports
    """
    try:
        if packet.haslayer(TCP):
            # For TCP packets, get payload after TCP header
            tcp_layer = packet[TCP]
            payload = bytes(tcp_layer.payload)
        elif packet.haslayer(UDP):
            # For UDP packets, get payload after UDP header  
            udp_layer = packet[UDP]
            payload = bytes(udp_layer.payload)
        else:
            # Fallback: get payload after IP header (IPv4 or IPv6)
            if packet.haslayer(IP):
                ip_layer = packet[IP]
                payload = bytes(ip_layer.payload)
            elif packet.haslayer(IPv6):
                ipv6_layer = packet[IPv6]
                payload = bytes(ipv6_layer.payload)
            else:
                return ""
            
        # Convert to hex string
        if len(payload) > 0:
            hex_payload = binascii.hexlify(payload).decode()
            return hex_payload
        else:
            return ""
            
    except Exception as e:
        print(f"Error extracting payload: {e}")
        return ""

def cut(obj, sec):
    """Cut object into sections"""
    if not obj or len(obj) == 0:
        return []
        
    result = [obj[i:i+sec] for i in range(0, len(obj), sec)]
    
    if not result or len(result) == 0:
        return []
        
    remanent_count = len(result[0]) % 4
    if remanent_count != 0:
        result = [obj[i:i+sec+remanent_count] for i in range(0, len(obj), sec+remanent_count)]
    
    return result

def bigram_generation(packet_datagram, packet_len=64):
    """Generate bigrams from packet data"""
    if not packet_datagram or len(packet_datagram) == 0:
        return ''
    
    result = ''
    generated_datagram = cut(packet_datagram, 1)
    
    if not generated_datagram or len(generated_datagram) == 0:
        return ''
    
    token_count = 0
    for sub_string_index in range(len(generated_datagram)):
        if sub_string_index != (len(generated_datagram) - 1):
            token_count += 1
            if token_count > packet_len:
                break
            else:
                merge_word_bigram = generated_datagram[sub_string_index] + generated_datagram[sub_string_index + 1]
        else:
            break
        result += merge_word_bigram
        result += ' '
    
    return result

def get_feature_packet_improved(pcap_file, payload_len=128):
    """
    Extract packet-level features with proper filtering and header removal
    Optimized for single packet per file (after splitcap)
    """
    try:
        packets = rdpcap(pcap_file)
        
        # Since each file contains one packet after splitcap
        if not packets or len(packets) == 0:
            return []
            
        # Filter the single packet
        filtered_packets = filter_packets(packets)
        
        if not filtered_packets:
            return []
        
        # Process the single valid packet
        packet = filtered_packets[0]
        payload_hex = extract_payload_only(packet)
        
        if not payload_hex:
            return []
        
        # Generate bigrams from payload only
        packet_data_string = bigram_generation(payload_hex, packet_len=payload_len)
        
        return [packet_data_string] if packet_data_string else []
        
    except Exception as e:
        print(f"Error extracting packet features from {pcap_file}: {e}")
        return []

def get_feature_flow_improved(pcap_file, payload_len=128, payload_pac=5):
    """
    Extract flow-level features with proper filtering and header removal
    Optimized for single flow per file (after splitcap)
    """
    try:
        packets = rdpcap(pcap_file)
        
        # Since each file contains one flow after splitcap
        if not packets or len(packets) == 0:
            return []
            
        # Filter all packets in the flow
        filtered_packets = filter_packets(packets)
        
        # Need at least 3 packets for flow (even after filtering)
        if len(filtered_packets) < 3:
            return []
        
        # Limit to payload_packet count
        packets_to_process = filtered_packets[:payload_pac]
        
        lengths = []
        times = []
        directions = []
        flow_data_string = ''
        
        for i, packet in enumerate(packets_to_process):
            payload_hex = extract_payload_only(packet)
            
            if payload_hex:
                bigram_data = bigram_generation(payload_hex, packet_len=payload_len)
                flow_data_string += bigram_data
                
                # Extract other flow features
                lengths.append(len(payload_hex) // 2)  # Convert hex to byte length
                times.append(float(packet.time))
                directions.append(1 if i % 2 == 0 else -1)
        
        if not flow_data_string:
            return []
        
        # Return: [payload, lengths, times, directions, message_types]
        return [flow_data_string, lengths, times, directions, []]
        
    except Exception as e:
        print(f"Error extracting flow features from {pcap_file}: {e}")
        return []

def clean_single_pcap(source_file, target_file):
    """
    Clean a single PCAP file by removing irrelevant packets
    """
    try:
        print(f"Cleaning: {source_file}")
        packets = rdpcap(source_file)
        filtered_packets = filter_packets(packets)
        
        if filtered_packets:
            wrpcap(target_file, filtered_packets)
            print(f"Saved {len(filtered_packets)} packets (from {len(packets)}) to {target_file}")
            return len(filtered_packets)
        else:
            print(f"No valid packets in {source_file}")
            return 0
            
    except Exception as e:
        print(f"Error processing {source_file}: {e}")
        return 0

def process_single_file(args):
    """
    Process a single PCAP file - designed for parallel execution
    Args: (pcap_file, dataset_level, payload_length, payload_packet)
    Returns: (success, pcap_file, features)
    """
    pcap_file, dataset_level, payload_length, payload_packet = args
    
    try:
        if dataset_level == "packet":
            features = get_feature_packet_improved(pcap_file, payload_len=payload_length)
        else:
            features = get_feature_flow_improved(pcap_file, payload_len=payload_length, payload_pac=payload_packet)
        
        if features and len(features) > 0:
            return True, pcap_file, features
        else:
            return False, pcap_file, None
            
    except Exception as e:
        print(f"Error processing {pcap_file}: {e}")
        return False, pcap_file, None

def process_pcap_dataset(input_path, output_path, samples_per_label=1000, 
                        dataset_level="packet", payload_length=128, payload_packet=5,
                        max_workers=None,dataset_name=""):
    """
    Complete dataset processing function for Kaggle with parallel processing
    
    Args:
        input_path: Path to input dataset with label folders
        output_path: Path to save processed dataset
        samples_per_label: Number of samples to extract per label
        dataset_level: "packet" or "flow"
        payload_length: Length for bigram generation
        payload_packet: Number of packets per flow (for flow level)
        max_workers: Number of parallel workers (None = auto-detect)
    
    Returns:
        dataset: Dictionary containing processed dataset
    """
    print(f"Processing dataset: {input_path}")
    print(f"Dataset level: {dataset_level}")
    print(f"Samples per label: {samples_per_label}")
    
    # Set max_workers if not specified
    if max_workers is None:
        max_workers = min(cpu_count(), 8)  # Don't use too many cores
    print(f"Using {max_workers} parallel workers")
    
    # Create output directory
    Path(output_path).mkdir(parents=True, exist_ok=True)
    
    # Get label folders
    label_folders = [d for d in os.listdir(input_path) 
                    if os.path.isdir(os.path.join(input_path, d))]
    
    print(f"Found labels: {label_folders}")
    
    dataset = {}
    label_id = {label: idx for idx, label in enumerate(label_folders)}
    
    processed_files = []
    
    for label_name in label_folders:
        print(f"\n{'='*50}")
        print(f"Processing label: {label_name}")
        print(f"{'='*50}")
        
        label_path = os.path.join(input_path, label_name)
        
        # Initialize dataset entry
        label_idx = label_id[label_name]
        if dataset_level == "packet":
            dataset[label_idx] = {"samples": 0, "payload": {}}
        else:  # flow
            dataset[label_idx] = {
                "samples": 0, "payload": {}, "length": {}, 
                "time": {}, "direction": {}, "message_type": {}
            }
        
        # Collect all PCAP files (each contains one packet/flow after splitcap)
        pcap_files = []
        for root, dirs, files in os.walk(label_path):
            for file in files:
                if file.lower().endswith(('.pcap', '.pcapng')):
                    pcap_files.append(os.path.join(root, file))
        
        if not pcap_files:
            print(f"No PCAP files found in {label_path}")
            continue
        
        print(f"Found {len(pcap_files)} PCAP files")
        
        # Sample files if needed (since each file = one sample after splitcap)
        files_to_process = (random.sample(pcap_files, min(samples_per_label, len(pcap_files))) 
                          if len(pcap_files) > samples_per_label else pcap_files)
        
        print(f"Processing {len(files_to_process)} files")
        
        # Prepare arguments for parallel processing
        args_list = [
            (pcap_file, dataset_level, payload_length, payload_packet)
            for pcap_file in files_to_process
        ]
        
        # Process files in parallel
        successful_extractions = 0
        
        with ProcessPoolExecutor(max_workers=max_workers) as executor:
            # Submit all tasks
            future_to_file = {
                executor.submit(process_single_file, args): args[0] 
                for args in args_list
            }
            
            # Process results as they complete
            for future in tqdm.tqdm(as_completed(future_to_file), 
                                   total=len(future_to_file), 
                                   desc=f"Processing {label_name}"):
                
                success, pcap_file, features = future.result()
                
                if success and features:
                    dataset[label_idx]["samples"] += 1
                    sample_key = str(dataset[label_idx]["samples"])
                    
                    # Store features
                    dataset[label_idx]["payload"][sample_key] = features[0]
                    
                    if dataset_level == "flow" and len(features) > 1:
                        dataset[label_idx]["length"][sample_key] = features[1]
                        dataset[label_idx]["time"][sample_key] = features[2]
                        dataset[label_idx]["direction"][sample_key] = features[3]
                        dataset[label_idx]["message_type"][sample_key] = features[4]
                    
                    processed_files.append(pcap_file)
                    successful_extractions += 1
        
        print(f"Label {label_name}: {dataset[label_idx]['samples']} samples extracted successfully")
        print(f"Success rate: {successful_extractions}/{len(files_to_process)} ({100*successful_extractions/len(files_to_process):.1f}%)")
    
    # Save results
    dataset_file = os.path.join(output_path, f"dataset_{dataset_name}.json")
    with open(dataset_file, "w") as f:
        json.dump(dataset, f, ensure_ascii=False, indent=4)
    
    # Save processed files list
    files_record = os.path.join(output_path, "processed_files_ustf-tfc.txt")
    with open(files_record, "w") as f:
        for file_path in processed_files:
            f.write(f"{file_path}\n")
    
    # Print statistics
    total_samples = 0
    print(f"\n{'='*50}")
    print("DATASET STATISTICS")
    print(f"{'='*50}")
    for label_name in label_folders:
        label_idx = label_id[label_name]
        samples = dataset[label_idx]["samples"]
        print(f"{label_idx}\t{label_name}\t{samples}")
        total_samples += samples
    print(f"Total samples: {total_samples}")
    
    return dataset

def clean_dataset_folder(input_path, output_path):
    """
    Clean all PCAP files in a dataset folder structure
    """
    print(f"Cleaning dataset: {input_path} -> {output_path}")
    
    cleaned_count = 0
    total_count = 0
    
    for root, dirs, files in os.walk(input_path):
        for file in files:
            if file.lower().endswith(('.pcap', '.pcapng')):
                total_count += 1
                source_path = os.path.join(root, file)
                
                # Maintain folder structure in output
                relative_path = os.path.relpath(root, input_path)
                output_subfolder = os.path.join(output_path, relative_path)
                Path(output_subfolder).mkdir(parents=True, exist_ok=True)
                
                # Create output filename
                clean_filename = f"clean_{file}"
                target_path = os.path.join(output_subfolder, clean_filename)
                
                # Clean the PCAP file
                try:
                    result = clean_single_pcap(source_path, target_path)
                    if result > 0:
                        cleaned_count += 1
                except Exception as e:
                    print(f"Error cleaning {source_path}: {e}")
    
    print(f"Cleaning complete: {cleaned_count}/{total_count} files successfully cleaned")
    return output_path


## transform to tsv for flow dataset

In [ ]:
def clean_packet_payload(packet_payload, target_length=74):
    """
    Clean and standardize packet payload
    - Remove extra whitespace
    - Truncate or pad to target length
    - Handle empty/invalid payloads
    - Use 4-character hex groups (0000 format)
    """
    if not packet_payload or packet_payload.strip() == '':
        # Empty packet - pad with 0000 groups
        return ' '.join(['0000'] * target_length)
    
    # Split into tokens and clean
    tokens = packet_payload.strip().split()
    
    # Filter and normalize tokens to 4-character hex groups
    clean_tokens = []
    for token in tokens:
        if token == '<PAD>':
            clean_tokens.append('0000')  # Replace <PAD> with 0000
        elif len(token) == 4 and all(c in '0123456789abcdefABCDEF' for c in token):
            clean_tokens.append(token.lower())  # Normalize to lowercase
        elif len(token) == 2 and all(c in '0123456789abcdefABCDEF' for c in token):
            # Convert 2-char hex to 4-char by padding with zeros
            clean_tokens.append('00' + token.lower())
        elif token in ['00', '0']:
            clean_tokens.append('0000')  # Normalize short zeros
        else:
            # Skip invalid tokens or try to fix them
            if len(token) > 4:
                # Take first 4 characters if too long
                clean_tokens.append(token[:4].lower())
            else:
                # Pad with zeros if too short
                padded = token.ljust(4, '0').lower()
                clean_tokens.append(padded)
    
    # Truncate or pad to target length
    if len(clean_tokens) >= target_length:
        return ' '.join(clean_tokens[:target_length])
    else:
        # Pad with 0000 groups
        padding_needed = target_length - len(clean_tokens)
        padded_tokens = clean_tokens + ['0000'] * padding_needed
        return ' '.join(padded_tokens)

def process_flow_to_fixed_format(flow_payload, target_packets=5, packet_length=74):
    """
    Process flow payload to have exactly 5 packets of length 74 each
    Using 4-character hex groups (0000 format)
    """
    if not flow_payload or flow_payload.strip() == '':
        # Empty flow - create 5 empty packets with 0000 padding
        empty_packet = ' '.join(['0000'] * packet_length)
        return ' '.join([empty_packet] * target_packets)
    
    # Try to split into packets (assuming packets are separated by multiple spaces or newlines)
    # First, normalize separators
    normalized_payload = flow_payload.replace('\n', ' ').replace('\t', ' ')
    
    # Split into tokens
    all_tokens = normalized_payload.split()
    
    # Clean and normalize tokens to 4-character hex groups
    clean_tokens = []
    for token in all_tokens:
        if token == '<PAD>':
            clean_tokens.append('0000')
        elif len(token) == 4 and all(c in '0123456789abcdefABCDEF' for c in token):
            clean_tokens.append(token.lower())
        elif len(token) == 2 and all(c in '0123456789abcdefABCDEF' for c in token):
            # Convert 2-char to 4-char hex
            clean_tokens.append('00' + token.lower())
        elif token in ['00', '0']:
            clean_tokens.append('0000')
        else:
            # Try to fix other formats
            if len(token) > 4:
                clean_tokens.append(token[:4].lower())
            elif len(token) < 4 and all(c in '0123456789abcdefABCDEF' for c in token):
                clean_tokens.append(token.ljust(4, '0').lower())
            # Skip completely invalid tokens
    
    # Create packets
    packets = []
    tokens_per_packet = packet_length
    
    for i in range(target_packets):
        start_idx = i * tokens_per_packet
        end_idx = start_idx + tokens_per_packet
        
        if start_idx < len(clean_tokens):
            packet_tokens = clean_tokens[start_idx:end_idx]
            
            # Pad packet if needed with 0000
            if len(packet_tokens) < tokens_per_packet:
                padding_needed = tokens_per_packet - len(packet_tokens)
                packet_tokens.extend(['0000'] * padding_needed)
                
            packets.append(' '.join(packet_tokens))
        else:
            # No more tokens - create empty packet with 0000 padding
            empty_packet = ' '.join(['0000'] * tokens_per_packet)
            packets.append(empty_packet)
    
    # Join packets with a separator (e.g., ' | ')
    return ' | '.join(packets)

def analyze_dataset_structure(dataset_path):
    """
    Analyze the current dataset structure to understand the data
    """
    print("Analyzing dataset structure...")
    
    with open(dataset_path, 'r', encoding='utf-8') as f:
        dataset = json.load(f)
    
    stats = {
        'total_labels': len(dataset),
        'samples_per_label': {},
        'payload_length_stats': [],
        'common_tokens': Counter(),
        'empty_payloads': 0
    }
    
    for label_id, label_data in dataset.items():
        samples = label_data.get('samples', 0)
        stats['samples_per_label'][label_id] = samples
        
        # Analyze payloads
        for sample_id, payload in label_data.get('payload', {}).items():
            if not payload or payload.strip() == '':
                stats['empty_payloads'] += 1
                continue
                
            tokens = payload.split()
            stats['payload_length_stats'].append(len(tokens))
            
            # Count common tokens (sample first 1000 for speed)
            for token in tokens[:100]:
                stats['common_tokens'][token] += 1
    
    # Print analysis
    print(f"Dataset Analysis:")
    print(f"  Total labels: {stats['total_labels']}")
    print(f"  Samples per label: {dict(stats['samples_per_label'])}")
    print(f"  Empty payloads: {stats['empty_payloads']}")
    
    if stats['payload_length_stats']:
        print(f"  Payload length stats:")
        print(f"    Min: {min(stats['payload_length_stats'])} tokens")
        print(f"    Max: {max(stats['payload_length_stats'])} tokens") 
        print(f"    Mean: {np.mean(stats['payload_length_stats']):.1f} tokens")
        print(f"    Median: {np.median(stats['payload_length_stats']):.1f} tokens")
    
    print(f"  Most common tokens: {stats['common_tokens'].most_common(10)}")
    
    # Check if data is in 4-character hex format
    hex4_count = sum(1 for token, count in stats['common_tokens'].items() if len(token) == 4)
    hex2_count = sum(1 for token, count in stats['common_tokens'].items() if len(token) == 2)
    
    print(f"  Token format analysis:")
    print(f"    4-character hex tokens: {hex4_count}")
    print(f"    2-character hex tokens: {hex2_count}")
    print(f"    Detected format: {'4-char hex groups (0000)' if hex4_count > hex2_count else '2-char hex pairs (00)'}")
    
    return stats

def polish_dataset(input_dataset_path, output_tsv_path, target_packets=5, packet_length=74):
    """
    Polish the dataset and create final TSV file
    """
    print(f"Polishing dataset...")
    print(f"Target format: {target_packets} packets × {packet_length} tokens each")
    
    # Load dataset
    with open(input_dataset_path, 'r', encoding='utf-8') as f:
        dataset = json.load(f)
    
    # Prepare TSV data
    tsv_data = []
    total_samples = 0
    successful_samples = 0
    
    # Process each label
    for label_id, label_data in tqdm.tqdm(dataset.items(), desc="Processing labels"):
        samples = label_data.get('samples', 0)
        
        # Process each sample
        for sample_id, payload in label_data.get('payload', {}).items():
            total_samples += 1
            
            try:
                # Process payload to fixed format
                processed_payload = process_flow_to_fixed_format(
                    payload, target_packets=target_packets, packet_length=packet_length
                )
                
                # Add to TSV data
                tsv_data.append([label_id, processed_payload])
                successful_samples += 1
                
            except Exception as e:
                print(f"Error processing sample {sample_id} from label {label_id}: {e}")
                continue
    
    # Verify consistency
    print(f"Verifying consistency...")
    payload_lengths = [len(row[1].split()) for row in tsv_data]
    unique_lengths = set(payload_lengths)
    
    expected_length = target_packets * (packet_length + 1) + (target_packets - 1)  # +1 for spaces, separators
    
    print(f"Payload token counts: {unique_lengths}")
    print(f"Expected approximate length: ~{target_packets * packet_length} tokens")
    
    # Write TSV file
    print(f"Writing TSV file to {output_tsv_path}...")
    with open(output_tsv_path, 'w', newline='', encoding='utf-8') as f:
        tsv_writer = csv.writer(f, delimiter='\t')
        
        # Write header
        tsv_writer.writerow(['label', 'text_a'])
        
        # Write data
        for row in tsv_data:
            tsv_writer.writerow(row)
    
    print(f"Dataset polishing completed!")
    print(f"  Total samples processed: {total_samples}")
    print(f"  Successful samples: {successful_samples}")
    print(f"  Output file: {output_tsv_path}")
    
    return successful_samples

def validate_final_dataset(tsv_path, sample_size=10):
    """
    Validate the final dataset and show samples
    """
    print(f"Validating final dataset...")
    
    # Read TSV
    with open(tsv_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter='\t')
        rows = list(reader)
    
    if not rows:
        print("No data found in TSV file!")
        return
    
    # Check consistency
    payload_lengths = []
    label_counts = Counter()
    
    for row in rows:
        payload = row['text_a']
        label = row['label']
        
        # Count tokens (packets are separated by ' | ')
        packets = payload.split(' | ')
        total_tokens = sum(len(packet.split()) for packet in packets)
        payload_lengths.append(total_tokens)
        
        label_counts[label] += 1
    
    # Print validation results
    print(f"Validation Results:")
    print(f"  Total samples: {len(rows)}")
    print(f"  Labels distribution: {dict(label_counts)}")
    print(f"  Payload length consistency:")
    
    unique_lengths = set(payload_lengths)
    if len(unique_lengths) == 1:
        print(f"    ✓ All payloads have consistent length: {list(unique_lengths)[0]} tokens")
    else:
        print(f"    ⚠ Payload lengths vary: {unique_lengths}")
        print(f"    Min: {min(payload_lengths)}, Max: {max(payload_lengths)}")
    
    # Show samples
    print(f"\nSample entries:")
    for i, row in enumerate(rows[:sample_size]):
        payload = row['text_a']
        label = row['label']
        packets = payload.split(' | ')
        
        print(f"  Sample {i+1} (Label {label}):")
        print(f"    Packets: {len(packets)}")
        for j, packet in enumerate(packets):
            tokens = packet.split()
            print(f"    Packet {j+1}: {len(tokens)} tokens - {' '.join(tokens[:10])}{'...' if len(tokens) > 10 else ''}")
        print()

# Main execution
def main():
    # Configuration
    if not os.path.exists("/kaggle/working/tsv_file"):
        os.makedirs("tsv_file")
    input_dataset_path = "dataset.json"  # Path to your generated dataset
    output_tsv_path = "/kaggle/working/tsv_file/dataset_cicio.tsv"  # Output TSV file
    
    target_packets = 5      # Exactly 5 packets per flow
    packet_length = 74      # Exactly 74 tokens per packet
    
    print("="*60)
    print("DATASET POLISHING FOR BERT FINE-TUNING")
    print("="*60)
    
    # Step 1: Analyze current dataset
    if os.path.exists(input_dataset_path):
        stats = analyze_dataset_structure(input_dataset_path)
    else:
        print(f"Error: Dataset file {input_dataset_path} not found!")
        return
    
    print("\n" + "="*60)
    
    # Step 2: Polish and create TSV using correct packet-based approach
    successful_samples = polish_dataset(
        input_dataset_path=input_dataset_path,
        output_tsv_path=output_tsv_path,
        target_packets=target_packets,
        packet_length=packet_length
    )
    
    print("\n" + "="*60)
    
    # Step 3: Validate final result
    if os.path.exists(output_tsv_path):
        validate_final_dataset(output_tsv_path, sample_size=5)
    
    print("\n" + "="*60)
    print("POLISHING COMPLETED!")
    print("="*60)
    print(f"Your dataset is now ready for BERT fine-tuning: {output_tsv_path}")
    print(f"Format: {target_packets} packets × {packet_length} hex groups (4-char each)")
    print(f"Padding: 0000 (4-character hex groups)")
    print(f"All samples have consistent length for efficient training!")
    print(f"Example format: 000c 0c62 62c3 c3ce ... | 1234 5678 9abc def0 ... | ...")

if __name__ == "__main__":
    main()

## applied function

### Split pcap packet level

In [ ]:
result_packet = process_pcaps_packet_level_kaggle(
        input_path="/kaggle/input/pcap-data/Tor/Tor",
        output_path="/kaggle/working/tor", 
        samples_per_label=5000
    )
print("Processing complete!")
print(f"Results: {results}")

In [ ]:
result_packet = process_pcaps_packet_level_kaggle(
        input_path="/kaggle/input/pcap-data/CICIot2022/CICIot2022",
        output_path="/kaggle/working/packet_level_split/CICIot2022", 
        samples_per_label=5000
    )
print("Processing complete!")
print(f"Results: {results}")

In [ ]:
result_packet = process_pcaps_packet_level_kaggle(
        input_path="/kaggle/working/ISCX_VPN_pcap",
        output_path="/kaggle/working/packet_level_split/ISCX_VPN", 
        samples_per_label=5000
    )
print("Processing complete!")
print(f"Results: {results}")

In [ ]:
result_packet = process_pcaps_packet_level_kaggle(
        input_path="/kaggle/working/organized_ustf_tfc",
        output_path="/kaggle/working/packet_level_split/ustf-tfc", 
        samples_per_label=5000
    )
print("Processing complete!")
print(f"Results: {results}")

In [ ]:
!mkdir "/kaggle/working/packet_level_split/ustf-tfc"

### Generation of dataset json file

In [ ]:
# generated datasetjson file 
dataset = process_pcap_dataset(
    input_path="/kaggle/working/tor",
    output_path="/kaggle/working/dataset",
    samples_per_label=5000,
    dataset_level="packet",
    payload_length=128,
    payload_packet=5,
    max_workers=4  # Custom worker count
)

In [ ]:
# generated datasetjson file 
dataset = process_pcap_dataset(
    input_path="/kaggle/working/packet_level_split/CICIot2022",
    output_path="/kaggle/working/dataset",
    dataset_name="ciciot_packet",
    samples_per_label=5000,
    dataset_level="packet",
    payload_length=128,
    payload_packet=5,
    max_workers=4  # Custom worker count
)

In [ ]:
# generated datasetjson file 
dataset = process_pcap_dataset(
    input_path="/kaggle/working/packet_level_split/ISCX_VPN",
    output_path="/kaggle/working/dataset",
    dataset_name="iscxVpn_packet",
    samples_per_label=5000,
    dataset_level="packet",
    payload_length=128,
    payload_packet=5,
    max_workers=4  # Custom worker count
)

In [ ]:
# generated datasetjson file 
dataset = process_pcap_dataset(
    input_path="/kaggle/working/packet_level_split/ustf-tfc",
    output_path="/kaggle/working/dataset",
    dataset_name="ustf-tfc_packet",
    samples_per_label=5000,
    dataset_level="packet",
    payload_length=128,
    payload_packet=5,
    max_workers=4  # Custom worker count
)

### generation of tsv file for packet level 

In [ ]:
# generation tsv file fo json dataset
labels = ["0", "1", "2", "3", "4", "5","6","7","8","9","10","11","12","13","14","15","16","17","18","19"]

def normalize_payload(payload, length=640):
    """Ensure payload is exactly `length` characters by padding or trimming."""
    pad_unit = " 0000"
    while len(payload) < length:
        payload += pad_unit
    return payload[:length]  # trim if it exceeds

# Load dataset
with open("/kaggle/working/dataset/dataset_ustf-tfc_packet.json", "r") as file:
    dataset = json.load(file)

# Write TSV
with open("/kaggle/working/tsv_file/dataset_ustf-tfc_packet.tsv", "w", newline='') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')
    writer.writerow(["label", "text_a"])  # Header

    for label in labels:
        # Get payloads from JSON
        payloads = list(dataset[label]["payload"].values())

        for payload in payloads:
            normalized = normalize_payload(payload)
            writer.writerow([label, normalized])


In [ ]:
#generation tsv files for npy format dataset

# Load .npy files
dataset_save_path = "/kaggle/input/etbert-dataset/USTC-TFC_dataset/"

x_payload_train = np.load(dataset_save_path + "x_payload_train.npy", allow_pickle=True)
x_payload_test = np.load(dataset_save_path + "x_payload_test.npy", allow_pickle=True)
x_payload_valid = np.load(dataset_save_path + "x_payload_valid.npy", allow_pickle=True)

y_train = np.load(dataset_save_path + "y_train.npy", allow_pickle=True)
y_test = np.load(dataset_save_path + "y_test.npy", allow_pickle=True)
y_valid = np.load(dataset_save_path + "y_valid.npy", allow_pickle=True)

# Merge all data
x_all = np.concatenate([x_payload_train, x_payload_test, x_payload_valid], axis=0)
y_all = np.concatenate([y_train, y_test, y_valid], axis=0)

# Write to TSV
output_tsv_path = "/kaggle/working/tsv_file/dataset_tfc_packet_l.tsv"
with open(output_tsv_path, "w", newline='') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')
    writer.writerow(["label", "text_a"])  # Header

    for payload, label in zip(x_all, y_all):
        writer.writerow([str(label), payload])


## Other

In [ ]:
# Organize ISXVPN dataset pcap
results = organize_kaggle_pcaps()

In [ ]:
#Organize ustf-tfc dataset pcap
results = organize_ustf_tfc_kaggle()

# finetuning etBert

## etBert

In [ ]:
#fine_tuning the pretrained_model for ustf-tfc
!python3 /kaggle/input/et-bert/run_classifier_copy.py --pretrained_model_path /kaggle/working/finetuned_models/finetuned_model_ustf-tfc_l1.bin  \
                                   --vocab_path /kaggle/input/etbert-model/ET-BERT/models/encryptd_vocab.txt \
                                   --config_path /kaggle/input/etbert-model/ET-BERT/models/bert_base_config.json\
                                   --output_model_path /kaggle/working/finetuned_models/finetuned_model_ustf-tfc_l.bin\
                                   --output_path_confusion_matrix  /kaggle/working/confusion_matrix/conf_mat_ustf-tfc_l.csv\
                                    --output_result /kaggle/working/result_json/result_ustf-tfc_l.json\
                                   --train_path /kaggle/working/tsv_file_split/dataset_tfc_packet_l_train.tsv\
                                   --dev_path /kaggle/working/tsv_file_split/dataset_tfc_packet_l_test.tsv\
                                   --test_path /kaggle/working/tsv_file_split/dataset_tfc_packet_l_test.tsv\
                                    --test_only true \
                                   --epochs_num 10 --batch_size 32 --embedding word_pos_seg \
                                   --encoder transformer --mask fully_visible \
                                   --seq_length 128 --learning_rate 2e-5
                                   

In [ ]:
!ls

In [ ]:
#fine_tuning the pretrained_model for ustf-tfc
!python3  /kaggle/input/et-bert/run_classifier_copy.py --pretrained_model_path /kaggle/working/finetuned_models/finetuned_model_ustf-tfc_l1.bin \
                                   --vocab_path /kaggle/input/etbert-model/ET-BERT/models/encryptd_vocab.txt \
                                   --config_path /kaggle/input/etbert-model/ET-BERT/models/bert_base_config.json\
                                   --output_model_path /kaggle/working/finetuned_models/finetuned_model_ustf-tfc2.bin\
                                   --output_path_confusion_matrix  /kaggle/working/confusion_matrix/conf_mat_ustf-tfc1.csv\
                                   --train_path /kaggle/working/tsv_file_split/dataset_ustf-tfc_packet_train.tsv\
                                   --dev_path /kaggle/working/tsv_file_split/dataset_ustf-tfc_packet_test.tsv\
                                   --test_path /kaggle/working/tsv_file_split/dataset_ustf-tfc_packet_test.tsv\
                                   --epochs_num 6 --batch_size 32 --embedding word_pos_seg \
                                   --encoder transformer --mask fully_visible \
                                   --seq_length 128 --learning_rate 2e-5
                                   

In [ ]:
!mkdir finetuned_models

In [ ]:
#fine_tuning the pretrained_model for isxTor
!python3  /kaggle/input/et-bert/run_classifier_copy.py --pretrained_model_path /kaggle/working/finetuned_models/finetuned_model_isxtor2.bin \
                                   --vocab_path /kaggle/input/etbert-model/ET-BERT/models/encryptd_vocab.txt \
                                   --config_path /kaggle/input/etbert-model/ET-BERT/models/bert_base_config.json\
                                   --output_model_path /kaggle/working/finetuned_models/finetuned_model_isxtor3.bin\
                                   --output_path_confusion_matrix  /kaggle/working/confusion_matrix/conf_mat_isxtor2.csv\
                                  --output_result /kaggle/working/result_json/result_isxtor.json\
                                   --train_path /kaggle/working/tsv_file_split/dataset_tor_packet_train.tsv\
                                   --dev_path /kaggle/working/tsv_file_split/dataset_tor_packet_test.tsv\
                                   --test_path /kaggle/working/tsv_file_split/dataset_tor_packet_test.tsv\
                                   --epochs_num 10 --batch_size 32 --embedding word_pos_seg \
                                   --encoder transformer --mask fully_visible \
                                   --seq_length 128 --learning_rate 2e-5

In [ ]:
#fine_tuning the pretrained_model for isxTor
!python3  /kaggle/input/et-bert/run_classifier_copy.py --pretrained_model_path /kaggle/input/pretrained-model/pretrain_model/etBert_pretrained_model.bin\
                                   --vocab_path /kaggle/input/etbert-model/ET-BERT/models/encryptd_vocab.txt \
                                   --config_path /kaggle/input/etbert-model/ET-BERT/models/bert_base_config.json\
                                   --output_model_path /kaggle/working/finetuned_models/finetuned_model_isxtor1.bin\
                                   --output_path_confusion_matrix  /kaggle/working/confusion_matrix/conf_mat_isxtor.csv\
                                  --output_result /kaggle/working/result_json/result_isxtor.json\
                                   --train_path /kaggle/working/tsv_file_split/dataset_tor_packet_train.tsv\
                                   --dev_path /kaggle/working/tsv_file_split/dataset_tor_packet_test.tsv\
                                   --test_path /kaggle/working/tsv_file_split/dataset_tor_packet_test.tsv\
                                   --epochs_num 5 --batch_size 16 --embedding word_pos_seg \
                                   --encoder transformer --mask fully_visible \
                                   --seq_length 512 --learning_rate 2e-5

In [ ]:
#fine_tuning the pretrained_model for isxTor
!python3  /kaggle/input/et-bert/run_classifier_copy.py --pretrained_model_path /kaggle/working/finetuned_models/finetuned_model_isxtor3.bin \
                                   --vocab_path /kaggle/input/etbert-model/ET-BERT/models/encryptd_vocab.txt \
                                   --config_path /kaggle/input/etbert-model/ET-BERT/models/bert_base_config.json\
                                   --output_model_path /kaggle/working/finetuned_models/finetuned_model_isxtor4.bin\
                                   --output_path_confusion_matrix  /kaggle/working/confusion_matrix/conf_mat_isxtor4.csv\
                                  --output_result /kaggle/working/result_json/result_isxtor4.json\
                                   --train_path /kaggle/working/tsv_file_split/dataset_tor_packet_train.tsv\
                                   --dev_path /kaggle/working/tsv_file_split/dataset_tor_packet_test.tsv\
                                   --test_path /kaggle/working/tsv_file_split/dataset_tor_packet_test.tsv\
                                   --epochs_num 15 --batch_size 32 --embedding word_pos_seg \
                                   --encoder transformer --mask fully_visible \
                                   --seq_length 128 --learning_rate 2e-5

In [ ]:
#fine_tuning the pretrained_model for isxvpn
!python3  /kaggle/input/et-bert/run_classifier_copy.py --pretrained_model_path /kaggle/input/pretrained-model/pretrain_model/etBert_pretrained_model.bin\
                                   --vocab_path /kaggle/input/etbert-model/ET-BERT/models/encryptd_vocab.txt \
                                   --config_path /kaggle/input/etbert-model/ET-BERT/models/bert_base_config.json\
                                   --output_model_path /kaggle/working/finetuned_models/finetuned_model_iscxvpn.bin\
                                   --output_path_confusion_matrix  /kaggle/working/confusion_matrix/conf_mat_iscxvpn.csv\
                                  --output_result /kaggle/working/result_json/result_iscxvpn.json\
                                   --train_path /kaggle/working/tsv_file_split/dataset_iscxvpn_packet_train.tsv\
                                   --dev_path /kaggle/working/tsv_file_split/dataset_iscxvpn_packet_test.tsv\
                                   --test_path /kaggle/working/tsv_file_split/dataset_iscxvpn_packet_test.tsv\
                                   --epochs_num 15 --batch_size 32 --embedding word_pos_seg \
                                   --encoder transformer --mask fully_visible \
                                   --seq_length 128 --learning_rate 2e-5

In [1]:
import os
import numpy as np
import pandas as pd

In [ ]:
df5=pd.read_csv("/kaggle/working/tsv_file_split/dataset_iscxvpn_packet_train.tsv", sep="\t")

In [27]:
X_test=np.load("/kaggle/input/etbert-dataset/ISCX-VPN_Service_dataset/x_payload_test.npy")
X_train=np.load("/kaggle/input/etbert-dataset/ISCX-VPN_Service_dataset/x_payload_train.npy")
X_valid=np.load("/kaggle/input/etbert-dataset/ISCX-VPN_Service_dataset/x_payload_valid.npy")
y_train=np.load("/kaggle/input/etbert-dataset/ISCX-VPN_Service_dataset/y_train.npy")
y_test=np.load("/kaggle/input/etbert-dataset/ISCX-VPN_Service_dataset/y_test.npy")
y_valid=np.load("/kaggle/input/etbert-dataset/ISCX-VPN_Service_dataset/y_valid.npy")
data_train=pd.DataFrame({"label":y_train,"text_a":X_train})
data_test=pd.DataFrame({"label":y_test,"text_a":X_test})
data_valid=pd.DataFrame({"label":y_valid,"text_a":X_valid})
data=pd.concat([data_train,data_test,data_valid])
df1=data[data["label"].isin([0,1,2,3,4,5])]

In [28]:
X_test=np.load("/kaggle/input/etbert-dataset/ISCX-VPN_app_dataset/x_payload_test.npy")
X_train=np.load("/kaggle/input/etbert-dataset/ISCX-VPN_app_dataset/x_payload_train.npy")
X_valid=np.load("/kaggle/input/etbert-dataset/ISCX-VPN_app_dataset/x_payload_valid.npy")
y_train=np.load("/kaggle/input/etbert-dataset/ISCX-VPN_app_dataset/y_train.npy")
y_test=np.load("/kaggle/input/etbert-dataset/ISCX-VPN_app_dataset/y_test.npy")
y_valid=np.load("/kaggle/input/etbert-dataset/ISCX-VPN_app_dataset/y_valid.npy")
data_train=pd.DataFrame({"label":y_train,"text_a":X_train})
data_test=pd.DataFrame({"label":y_test,"text_a":X_test})
data_valid=pd.DataFrame({"label":y_valid,"text_a":X_valid})
data=pd.concat([data_train,data_test,data_valid])
df2=data[data["label"]==10]

#data.to_csv("/kaggle/working/tsv_file/dataset_iscxvpn_packet.tsv",sep="\t",index=False)

In [22]:
X_test=np.load("/kaggle/input/etbert-dataset/USTC-TFC_dataset/x_payload_test.npy")
X_train=np.load("/kaggle/input/etbert-dataset/USTC-TFC_dataset/x_payload_train.npy")
X_valid=np.load("/kaggle/input/etbert-dataset/USTC-TFC_dataset/x_payload_valid.npy")
y_train=np.load("/kaggle/input/etbert-dataset/USTC-TFC_dataset/y_train.npy")
y_test=np.load("/kaggle/input/etbert-dataset/USTC-TFC_dataset/y_test.npy")
y_valid=np.load("/kaggle/input/etbert-dataset/USTC-TFC_dataset/y_valid.npy")
data_train=pd.DataFrame({"label":y_train,"text_a":X_train})
data_test=pd.DataFrame({"label":y_test,"text_a":X_test})
data_valid=pd.DataFrame({"label":y_valid,"text_a":X_valid})
data=pd.concat([data_train,data_test,data_valid])
data["label"].value_counts()
data.to_csv("/kaggle/working/tsv_file/dataset_ustf-tfc_packet_l.tsv",sep="\t",index=False)

In [29]:
df=pd.concat([df1,df2])
df.loc[df["label"]==10,"label"]=6

In [30]:
df.label.value_counts()

label
2    5000
4    5000
3    5000
5    5000
0    5000
1    5000
6    5000
Name: count, dtype: int64

In [31]:
df.to_csv("/kaggle/working/tsv_file/dataset_iscxvpn_packet_l.tsv",sep="\t",index=False)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the TSV file
data = pd.read_csv('/kaggle/working/tsv_file/dataset_tfc_packet_l.tsv', sep='\t')
print(len(data))

# Split into features (X) and labels (y)
X = data['text_a']
y = data['label']

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Optionally combine into DataFrames
train_df = pd.DataFrame({'label': y_train, 'text_a': X_train})
test_df = pd.DataFrame({'label': y_test, 'text_a': X_test})

# Save to new TSV files
train_df.to_csv('tsv_file_split/dataset_tfc_packet_l_train.tsv', sep='\t', index=False)
test_df.to_csv('tsv_file_split/dataset_tfc_packet_l_test.tsv', sep='\t', index=False)


In [ ]:
# Load the TSV file
import pandas as pd
train = pd.read_csv('/kaggle/working/tsv_file/dataset_tor_packet.tsv', sep='\t')
trai = pd.read_csv('/kaggle/working/tsv_file/dataset_tor.tsv', sep='\t')
train["label"].value_counts()
len(train['text_a'][0]),len(trai['text_a'][0])


# lstm models

In [2]:
from gensim.models import Word2Vec
import pandas as pd
import os
import numpy as np

In [6]:
#creer une liste de word for the embegings
path1="/kaggle/working/tsv_file_split/dataset_ciciot_packet_train.tsv"
path2="/kaggle/working/tsv_file_split/dataset_tfc_packet_l_test.tsv"
path3="/kaggle/working/tsv_file_split/dataset_tfc_packet_l_train.tsv"
path4="/kaggle/working/tsv_file_split/dataset_tor_packet_test.tsv"
path5="/kaggle/working/tsv_file_split/dataset_tor_packet_train.tsv"
path6="/kaggle/working/tsv_file_split/dataset_ustf-tfc_packet_test.tsv"
path7="/kaggle/working/tsv_file_split/dataset_ustf-tfc_packet_train.tsv"

paths=[path1,path2,path3,path4,path5,path6,path7]
df=pd.read_csv(path1)
lt=[]
for path in paths:
    df=pd.read_csv(path,sep="\t")
    lt.append(df)
df=pd.concat(lt)
sentences=df.text_a.tolist()
list_word=[sentence.split() for sentence in sentences]


In [7]:
#lets train our Word2vec Model
model_emb=Word2Vec(list_word,vector_size=64,window=5,workers=4)
#essayons d'embed o word
model_emb.wv["4e98"]
#model_emb.save("word2vec_model")

array([ 0.5278087 ,  0.02324824, -0.30430895,  0.55657595, -1.2761108 ,
        1.3570802 ,  0.22196038, -0.05355455,  0.10258319,  0.28360704,
       -0.3542523 ,  0.10284703, -0.41621166,  0.009511  ,  0.78432107,
       -0.38037065,  0.26946986,  0.3376374 ,  0.28275222, -0.46227115,
       -1.2999579 , -0.03900266, -0.06075571,  0.97147536,  0.7425491 ,
        0.6637179 ,  0.7277807 , -0.2122171 ,  1.1894776 ,  0.26970965,
       -0.46994507, -0.37154552,  0.32101426, -0.08506291, -0.95035744,
       -0.35429162,  0.24093682, -0.18618672,  0.05498379, -0.18210594,
        0.00682116, -0.7330861 ,  0.2582237 , -0.20799921,  0.5713041 ,
        0.02655398, -0.8263295 ,  0.49343908, -0.82526684, -0.6561987 ,
        1.088343  , -0.30323336, -0.12514731,  0.44312707,  0.10751735,
       -0.03526834,  0.39908347,  0.42476025,  0.12340546,  0.5504348 ,
        0.501027  , -0.16386531, -0.6997198 ,  0.06461726], dtype=float32)

In [8]:
model_emb=Word2Vec.load("/kaggle/working/word2vec_model")

In [9]:
#now je dois creer un dataframe ou store les sentences et words sous forme de vecteur
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import json
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
#construction du model lstm
from tensorflow.keras.models import Sequential
from tensorflow.keras import utils
from tensorflow.keras.metrics import Precision, Recall
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, LSTM, Dense, Dropout, 
    MultiHeadAttention, LayerNormalization, 
    GlobalAveragePooling1D, Bidirectional
)


2025-09-28 07:46:21.533385: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759045581.766541      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759045581.841887      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [24]:
#take the dataset in a pandas dataframe 
df_iscx_vpn=pd.read_csv("/kaggle/working/tsv_file/dataset_iscx_vpn_packet.tsv",sep="\t")
#df_ciciot=pd.read_csv("/kaggle/working/tsv_file/dataset_ciciot_packet.tsv",sep="\t")
df_ustf_tfc=pd.read_csv("/kaggle/working/tsv_file/dataset_ustf-tfc_packet.tsv",sep="\t")
#df_tor=pd.read_csv("/kaggle/working/tsv_file/dataset_tor_packet.tsv",sep="\t")

In [25]:
df_ustf_tfc["label"].value_counts()

label
0     5000
1     5000
2     5000
3     5000
4     5000
5     5000
7     5000
8     5000
9     4988
10    1389
6       62
Name: count, dtype: int64

In [10]:
def training_prep(df:pd.DataFrame):
    tokenizer=Tokenizer()
    tokenizer.fit_on_texts(df_iscx_vpn.text_a)
    X_train=df.text_a
    X_test=df.text_a
    y_train=df.label
    y_test=df.label
    
    #faire une liste d'index des mots par le plus frequent au moins frequent
    X_train_seq=tokenizer.texts_to_sequences(X_train)
    X_test_seq=tokenizer.texts_to_sequences(X_test)
    
    # Padding des séquences pour avoir une longueur fixe
    max_length = max(len(seq) for seq in X_train_seq)
    X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
    X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post')
    embedding_matrice=np.zeros((len(tokenizer.word_index)+1,model_emb.vector_size))
    for word,i in tokenizer.word_index.items():
        if word in model_emb.wv:
            embedding_matrice[i]=model_emb.wv[word]
            
    nbre_classes=df.label.value_counts().count()
    #transformer mes label avec one_hot_encoding
    y_train_onehot=utils.to_categorical(y_train,num_classes=nbre_classes)
    y_test_onehot=utils.to_categorical(y_test,num_classes=nbre_classes)

    result= {
    "nbre_classes": nbre_classes,
    "tokenizer": tokenizer,
    "embedding_matrice": embedding_matrice,
    "max_length": max_length,
    "X_train_pad": X_train_pad,
    "X_test_pad": X_test_pad,
    "y_train_onehot": y_train_onehot,
    "y_test_onehot":y_test_onehot
    }

    
    return result
    
def evaluation(model, result,saving_path):
    # Get predictions
    pred_prob = model.predict(result["X_test_pad"],batch_size=32)
    y_pred = np.argmax(pred_prob, axis=1)
    y_true = np.argmax(result["y_test_onehot"], axis=1)
    
    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average='weighted'
    )
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Create results
    results = {
        "acc": float(accuracy),
        "weighted_pre": float(precision),
        "weighted_rec": float(recall),
        "weighted_f1": float(f1),
        "cm": cm.tolist()
    }
    
    with open(f"lsmt_results/{saving_path}.json","w") as file:
        json.dump(results,file, indent=2)
    # Print and return JSON
    print(json.dumps(results, indent=2))
    return results
    


In [33]:
model_lstm=Sequential()
model_lstm.add(Embedding(input_dim=result["embedding_matrice"].shape[0],
                          output_dim=result["embedding_matrice"].shape[1],
                          weights=[result["embedding_matrice"]],
                          input_length=result["max_length"] ,
                          trainable=False
                         ))
model_lstm.add(LSTM(128,return_sequences=True))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(64,return_sequences=True))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(32))
model_lstm.add(Dense(result["nbre_classes"],activation="softmax"))

model_lstm.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

model_lstm.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │       4,194,368 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_12 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_13 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_14 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,194,368 (16.00 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 4,194,368 (16.00 MB)

In [29]:
model_lstm.fit(X_train_pad, y_train_onehot, epochs=20, batch_size=64, validation_split=0.2)

Epoch 1/20
353/353 ━━━━━━━━━━━━━━━━━━━━ 133s 349ms/step - accuracy: 0.4459 - loss: 1.2880 - val_accuracy: 0.0067 - val_loss: 6.2681
Epoch 2/20
353/353 ━━━━━━━━━━━━━━━━━━━━ 121s 342ms/step - accuracy: 0.5200 - loss: 1.0923 - val_accuracy: 0.0484 - val_loss: 6.6255
Epoch 3/20
353/353 ━━━━━━━━━━━━━━━━━━━━ 118s 335ms/step - accuracy: 0.6244 - loss: 0.8602 - val_accuracy: 0.0510 - val_loss: 8.0539
Epoch 4/20
353/353 ━━━━━━━━━━━━━━━━━━━━ 118s 336ms/step - accuracy: 0.6436 - loss: 0.8097 - val_accuracy: 0.0758 - val_loss: 8.4561
Epoch 5/20
353/353 ━━━━━━━━━━━━━━━━━━━━ 123s 349ms/step - accuracy: 0.6817 - loss: 0.7315 - val_accuracy: 0.0760 - val_loss: 8.3994
Epoch 6/20
353/353 ━━━━━━━━━━━━━━━━━━━━ 118s 335ms/step - accuracy: 0.6799 - loss: 0.7291 - val_accuracy: 0.0468 - val_loss: 8.6970
Epoch 7/20
353/353 ━━━━━━━━━━━━━━━━━━━━ 119s 338ms/step - accuracy: 0.6832 - loss: 0.7189 - val_accuracy: 0.0746 - val_loss: 8.7641
Epoch 8/20
353/353 ━━━━━━━━━━━━━━━━━━━━ 118s 333ms/step - accuracy: 0.6889 -

In [ ]:
#loading the model
# Dummy input (batch size 1, sequence length = max_length)
dummy_input = np.zeros((1, result["max_length"]), dtype=np.int32)
model_lstm(dummy_input)  # This builds the model
model_lstm.load_weights("lstm_models/lstm_base_iscx_vpn.weights.h5")

In [33]:
#model_lstm.save_weights("lstm_models/lstm_base_iscx_vpn.weights.h5")

## lstm+attention

In [11]:
def create_lstm_attention_model(embedding_matrice, max_length, nbre_classes):
    """
    Creates LSTM + Multi-Head Attention model for network classification
    
    Args:
        embedding_matrice: Pre-trained embedding matrix
        max_length: Maximum sequence length
        nbre_classes: Number of output classes
    
    Returns:
        Compiled Keras model
    """
    inputs = Input(shape=(max_length,))
    
    # Embedding layer (frozen pre-trained embeddings)
    embedding = Embedding(
        input_dim=embedding_matrice.shape[0],
        output_dim=embedding_matrice.shape[1],
        weights=[embedding_matrice],
        input_length=max_length,
        trainable=False
    )(inputs)
    
    # LSTM layers for sequential pattern extraction
    lstm1 = Bidirectional(LSTM(128, return_sequences=True))(embedding)
    lstm1 = Dropout(0.3)(lstm1)
    
    lstm2 = Bidirectional(LSTM(64, return_sequences=True))(lstm1)
    lstm2 = Dropout(0.3)(lstm2)
    
    # Multi-head attention on LSTM outputs
    attention = MultiHeadAttention(
        num_heads=8,           # 8 attention heads
        key_dim=64,           # Dimension of each head
        
        dropout=0.1
    )(lstm2, lstm2)           # Self-attention on LSTM outputs
    
    # Residual connection + Layer normalization
    attention_output = LayerNormalization()(attention + lstm2)
    
    # Global pooling to reduce sequence dimension
    pooled = GlobalAveragePooling1D()(attention_output)
    
    # Classification head
    dense1 = Dense(128, activation='relu')(pooled)
    dense1 = Dropout(0.5)(dense1)
    
    outputs = Dense(nbre_classes, activation='softmax')(dense1)
    
    # Create model
    model = Model(inputs=inputs, outputs=outputs)
    
    # Compile with same settings as your original + additional metrics
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy', Precision(), Recall()]
    )
    
    return model

In [12]:
# Optional: Training callbacks for better performance
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=50,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-15
    )
]

In [32]:
#take the dataset in a pandas dataframe 
df_iscx_vpn=pd.read_csv("/kaggle/working/tsv_file/dataset_iscxvpn_packet_l.tsv",sep="\t")
df_ciciot=pd.read_csv("/kaggle/working/tsv_file/dataset_ciciot_packet.tsv",sep="\t")
df_ustf_tfc=pd.read_csv("/kaggle/working/tsv_file/dataset_ustf-tfc_packet_l.tsv",sep="\t")
df_tor=pd.read_csv("/kaggle/working/tsv_file/dataset_tor_packet.tsv",sep="\t")

In [33]:
df_iscx_vpn["label"].value_counts()

label
2    5000
4    5000
3    5000
5    5000
0    5000
1    5000
6    5000
Name: count, dtype: int64

In [14]:
# Create your model
def model_initiation(df):
    result=training_prep(df)
    model_lstm_attention = create_lstm_attention_model(result["embedding_matrice"], result["max_length"], result["nbre_classes"])
    # Display architecture
    
    return model_lstm_attention,result

In [34]:
# Train the model iscx_vpn 
model_lstm_attention_iscx,result=model_initiation(df_iscx_vpn)
model_lstm_attention_iscx.summary()
history = model_lstm_attention_iscx.fit(
     result["X_train_pad"], result["y_train_onehot"],
     validation_split=0.2,
     epochs=50,
     batch_size=32,
     callbacks=callbacks
 )
model_lstm_attention_iscx.save_weights("lstm_models/lstm_plus_attention_iscx_vpn_l.weights.h5")

evaluation(model_lstm_attention_iscx,result,"model_lstm_attention_iscx_result_l")

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 64)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, 64, 64)         │      4,194,368 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_6           │ (None, 64, 256)        │        197,632 │ embedding_3[0][0]      │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_12 (Dropout)      │ (None, 64, 256)        │              0 │ bidirectional_6[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_7           │ (None, 64, 128)        │        164,352 │ dropout_12[0][0]       │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_13 (Dropout)      │ (None, 64, 128)        │              0 │ bidirectional_7[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_3    │ (None, 64, 128)        │        263,808 │ dropout_13[0][0],      │
│ (MultiHeadAttention)      │                        │                │ dropout_13[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_3 (Add)               │ (None, 64, 128)        │              0 │ multi_head_attention_… │
│                           │                        │                │ dropout_13[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_3     │ (None, 64, 128)        │            256 │ add_3[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 128)            │              0 │ layer_normalization_3… │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 128)            │         16,512 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_15 (Dropout)      │ (None, 128)            │              0 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 7)              │            903 │ dropout_15[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,837,831 (18.45 MB)

 Trainable params: 643,463 (2.45 MB)

 Non-trainable params: 4,194,368 (16.00 MB)

Epoch 1/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 29s 21ms/step - accuracy: 0.8567 - loss: 0.4187 - precision_3: 0.9041 - recall_3: 0.8167 - val_accuracy: 0.2737 - val_loss: 11.9618 - val_precision_3: 0.2764 - val_recall_3: 0.2733 - learning_rate: 0.0010
Epoch 2/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 17s 20ms/step - accuracy: 0.9585 - loss: 0.1173 - precision_3: 0.9684 - recall_3: 0.9497 - val_accuracy: 0.2756 - val_loss: 11.9166 - val_precision_3: 0.2765 - val_recall_3: 0.2750 - learning_rate: 0.0010
Epoch 3/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 17s 20ms/step - accuracy: 0.9677 - loss: 0.0837 - precision_3: 0.9771 - recall_3: 0.9604 - val_accuracy: 0.2767 - val_loss: 16.1389 - val_precision_3: 0.2829 - val_recall_3: 0.2760 - learning_rate: 0.0010
Epoch 4/50
875/875 ━━━━━━━━━━━━━━━━━━━━ 17s 20ms/step - accuracy: 0.9717 - loss: 0.0688 - precision_3: 0.9816 - recall_3: 0.9646 - val_accuracy: 0.2760 - val_loss: 11.0302 - val_precision_3: 0.2758 - val_recall_3: 0.2716 - learning_rate: 0.0010
Epoch 5/50
875/875 ━

{'acc': 0.8508,
 'weighted_pre': 0.7723857865992797,
 'weighted_rec': 0.8508,
 'weighted_f1': 0.8005426895983093,
 'cm': [[4941, 59, 0, 0, 0, 0, 0],
  [133, 4866, 0, 1, 0, 0, 0],
  [8, 2, 4981, 0, 0, 9, 0],
  [0, 0, 0, 5000, 0, 0, 0],
  [0, 0, 0, 0, 5000, 0, 0],
  [3, 0, 7, 0, 0, 4990, 0],
  [0, 0, 339, 0, 4639, 22, 0]]}

In [36]:
#import the model
model_lstm_attention_isxc,result=model_initiation(df_iscx_vpn)
model_lstm_attention_isxc.load_weights("lstm_models/lstm_plus_attention_iscx_vpn_l.weights.h5")
evaluation(model_lstm_attention_isxc,result,"model_lstm_attention_iscx_result_l_temp")

1094/1094 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step
{
  "acc": 0.8508,
  "weighted_pre": 0.7723857865992797,
  "weighted_rec": 0.8508,
  "weighted_f1": 0.8005426895983093,
  "cm": [
    [
      4941,
      59,
      0,
      0,
      0,
      0,
      0
    ],
    [
      133,
      4866,
      0,
      1,
      0,
      0,
      0
    ],
    [
      8,
      2,
      4981,
      0,
      0,
      9,
      0
    ],
    [
      0,
      0,
      0,
      5000,
      0,
      0,
      0
    ],
    [
      0,
      0,
      0,
      0,
      5000,
      0,
      0
    ],
    [
      3,
      0,
      7,
      0,
      0,
      4990,
      0
    ],
    [
      0,
      0,
      339,
      0,
      4639,
      22,
      0
    ]
  ]
}


{'acc': 0.8508,
 'weighted_pre': 0.7723857865992797,
 'weighted_rec': 0.8508,
 'weighted_f1': 0.8005426895983093,
 'cm': [[4941, 59, 0, 0, 0, 0, 0],
  [133, 4866, 0, 1, 0, 0, 0],
  [8, 2, 4981, 0, 0, 9, 0],
  [0, 0, 0, 5000, 0, 0, 0],
  [0, 0, 0, 0, 5000, 0, 0],
  [3, 0, 7, 0, 0, 4990, 0],
  [0, 0, 339, 0, 4639, 22, 0]]}

In [35]:
df_iscx_vpn.label.value_counts()

label
2    5000
4    5000
3    5000
5    5000
0    5000
1    5000
6    5000
Name: count, dtype: int64

In [33]:
# Train the model ustf_tfc
model_lstm_attention_ustf,result=model_initiation(df_ustf_tfc)
model_lstm_attention_ustf.summary()
history = model_lstm_attention_ustf.fit(
     result["X_train_pad"], result["y_train_onehot"],
     validation_split=0.2,
     epochs=50,
     batch_size=32,
     callbacks=callbacks
 )
model_lstm_attention_ustf.save_weights("lstm_models/lstm_plus_attention_ustf_tfc.weights.h5")

evaluation(model_lstm_attention_ustf,result,"model_lstm_attention_ustf_result")

I0000 00:00:1758969672.704338      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1758969672.705036      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 64)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 64, 64)         │      4,194,368 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ (None, 64, 256)        │        197,632 │ embedding[0][0]        │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 64, 256)        │              0 │ bidirectional[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_1           │ (None, 64, 128)        │        164,352 │ dropout[0][0]          │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 64, 128)        │              0 │ bidirectional_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 64, 128)        │        263,808 │ dropout_1[0][0],       │
│ (MultiHeadAttention)      │                        │                │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 64, 128)        │              0 │ multi_head_attention[… │
│                           │                        │                │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 64, 128)        │            256 │ add[0][0]              │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d  │ (None, 128)            │              0 │ layer_normalization[0… │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │         16,512 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 128)            │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 20)             │          2,580 │ dropout_3[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,839,508 (18.46 MB)

 Trainable params: 645,140 (2.46 MB)

 Non-trainable params: 4,194,368 (16.00 MB)

Epoch 1/50


I0000 00:00:1758969684.068599     131 cuda_dnn.cc:529] Loaded cuDNN version 90300


2428/2428 ━━━━━━━━━━━━━━━━━━━━ 64s 22ms/step - accuracy: 0.8629 - loss: 0.4242 - precision: 0.9343 - recall: 0.8148 - val_accuracy: 0.9608 - val_loss: 0.1054 - val_precision: 0.9636 - val_recall: 0.9583 - learning_rate: 0.0010
Epoch 2/50
2428/2428 ━━━━━━━━━━━━━━━━━━━━ 51s 21ms/step - accuracy: 0.9614 - loss: 0.0995 - precision: 0.9665 - recall: 0.9568 - val_accuracy: 0.9701 - val_loss: 0.0726 - val_precision: 0.9717 - val_recall: 0.9677 - learning_rate: 0.0010
Epoch 3/50
2428/2428 ━━━━━━━━━━━━━━━━━━━━ 51s 21ms/step - accuracy: 0.9718 - loss: 0.0733 - precision: 0.9743 - recall: 0.9692 - val_accuracy: 0.9724 - val_loss: 0.0728 - val_precision: 0.9757 - val_recall: 0.9675 - learning_rate: 0.0010
Epoch 4/50
2428/2428 ━━━━━━━━━━━━━━━━━━━━ 51s 21ms/step - accuracy: 0.9759 - loss: 0.0620 - precision: 0.9785 - recall: 0.9730 - val_accuracy: 0.9812 - val_loss: 0.0495 - val_precision: 0.9825 - val_recall: 0.9804 - learning_rate: 0.0010
Epoch 5/50
2428/2428 ━━━━━━━━━━━━━━━━━━━━ 51s 21ms/step - a

NameError: name 'model_lstm_attention_ustf_result' is not defined

In [17]:
#import the model
model_lstm_attention_ustf,result=model_initiation(df_ustf_tfc)
model_lstm_attention_ustf.load_weights("lstm_models/lstm_plus_attention_ustf_tfc.weights.h5")
evaluation(model_lstm_attention_ustf,result,"model_lstm_attention_ustf_result")

I0000 00:00:1759045728.397055     124 cuda_dnn.cc:529] Loaded cuDNN version 90300


3035/3035 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step
{
  "acc": 0.9976419708592905,
  "weighted_pre": 0.9976447622662122,
  "weighted_rec": 0.9976419708592905,
  "weighted_f1": 0.997641397462687,
  "cm": [
    [
      5000,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0
    ],
    [
      0,
      5000,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0
    ],
    [
      0,
      0,
      5000,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0
    ],
    [
      0,
      0,
      0,
      5000,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0
    ],
    [
      0,
      0,

{'acc': 0.9976419708592905,
 'weighted_pre': 0.9976447622662122,
 'weighted_rec': 0.9976419708592905,
 'weighted_f1': 0.997641397462687,
 'cm': [[5000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 5000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 5000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 5000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 5000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 5000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 1, 0, 4994, 3, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 1, 4999, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 5000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 1, 0, 0, 4871, 2, 0, 1, 0, 0, 0, 123, 0, 0, 2],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 4992, 0, 0, 0, 0, 0, 1, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5000, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2113, 0, 0, 

In [13]:
# Train the model ciciot 
model_lstm_attention_ciciot,result_ciciot=model_initiation(df_ciciot)
model_lstm_attention_ciciot.summary()
history = model_lstm_attention_ciciot.fit(
     result_ciciot["X_train_pad"], result_ciciot["y_train_onehot"],
     validation_split=0.2,
     epochs=50,
     batch_size=32,
     callbacks=callbacks
 )
model_lstm_attention_ciciot.save_weights("lstm_models/lstm_plus_attention_ciciot.weights.h5")

evaluation(model_lstm_attention_ciciot,result_ciciot,"model_lstm_attention_ciciot_result")

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 128)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 128, 64)        │      4,194,368 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_2           │ (None, 128, 256)       │        197,632 │ embedding_1[0][0]      │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_4 (Dropout)       │ (None, 128, 256)       │              0 │ bidirectional_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_3           │ (None, 128, 128)       │        164,352 │ dropout_4[0][0]        │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_5 (Dropout)       │ (None, 128, 128)       │              0 │ bidirectional_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_1    │ (None, 128, 128)       │        263,808 │ dropout_5[0][0],       │
│ (MultiHeadAttention)      │                        │                │ dropout_5[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 128, 128)       │              0 │ multi_head_attention_… │
│                           │                        │                │ dropout_5[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 128, 128)       │            256 │ add_1[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 128)            │              0 │ layer_normalization_1… │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 128)            │         16,512 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_7 (Dropout)       │ (None, 128)            │              0 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 6)              │            774 │ dropout_7[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,837,702 (18.45 MB)

 Trainable params: 643,334 (2.45 MB)

 Non-trainable params: 4,194,368 (16.00 MB)

Epoch 1/50


I0000 00:00:1754771282.588394     100 cuda_dnn.cc:529] Loaded cuDNN version 90300


749/749 ━━━━━━━━━━━━━━━━━━━━ 41s 38ms/step - accuracy: 0.8604 - loss: 0.3759 - precision_1: 0.8931 - recall_1: 0.8301 - val_accuracy: 0.1630 - val_loss: 9.9135 - val_precision_1: 0.1630 - val_recall_1: 0.1630 - learning_rate: 0.0010
Epoch 2/50
749/749 ━━━━━━━━━━━━━━━━━━━━ 27s 36ms/step - accuracy: 0.9725 - loss: 0.0787 - precision_1: 0.9731 - recall_1: 0.9721 - val_accuracy: 0.1620 - val_loss: 11.7892 - val_precision_1: 0.1620 - val_recall_1: 0.1620 - learning_rate: 0.0010
Epoch 3/50
749/749 ━━━━━━━━━━━━━━━━━━━━ 27s 36ms/step - accuracy: 0.9792 - loss: 0.0627 - precision_1: 0.9797 - recall_1: 0.9788 - val_accuracy: 0.1622 - val_loss: 12.5865 - val_precision_1: 0.1622 - val_recall_1: 0.1622 - learning_rate: 0.0010
Epoch 4/50
749/749 ━━━━━━━━━━━━━━━━━━━━ 27s 36ms/step - accuracy: 0.9802 - loss: 0.0634 - precision_1: 0.9804 - recall_1: 0.9795 - val_accuracy: 0.1630 - val_loss: 12.5362 - val_precision_1: 0.1634 - val_recall_1: 0.1630 - learning_rate: 0.0010
Epoch 5/50
749/749 ━━━━━━━━━━━━━

{'acc': 0.8100504527381469,
 'weighted_pre': 0.7104348352760586,
 'weighted_rec': 0.8100504527381469,
 'weighted_f1': 0.7490522788632759,
 'cm': [[4527, 0, 414, 0, 7, 0],
  [32, 4950, 0, 0, 0, 0],
  [162, 0, 4836, 0, 1, 0],
  [0, 2, 0, 4998, 0, 0],
  [0, 67, 0, 0, 4933, 0],
  [1503, 0, 3489, 0, 8, 0]]}

In [10]:
# Train the model tor
model_lstm_attention_tor,result_tor=model_initiation(df_tor)
model_lstm_attention_tor.summary()
history = model_lstm_attention_tor.fit(
     result_tor["X_train_pad"], result_tor["y_train_onehot"],
     validation_split=0.2,
     epochs=50,
     batch_size=32,
     callbacks=callbacks
 )
model_lstm_attention_tor.save_weights("lstm_models/lstm_plus_attention_tor.weights.h5")

evaluation(model_lstm_attention_tor,result_tor,"model_lstm_attention_tor_result")

I0000 00:00:1754826105.216983      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1754826105.217744      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 128)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 128, 64)        │      4,194,368 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ (None, 128, 256)       │        197,632 │ embedding[0][0]        │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 128, 256)       │              0 │ bidirectional[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_1           │ (None, 128, 128)       │        164,352 │ dropout[0][0]          │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 128, 128)       │              0 │ bidirectional_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 128, 128)       │        263,808 │ dropout_1[0][0],       │
│ (MultiHeadAttention)      │                        │                │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 128, 128)       │              0 │ multi_head_attention[… │
│                           │                        │                │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 128, 128)       │            256 │ add[0][0]              │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d  │ (None, 128)            │              0 │ layer_normalization[0… │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │         16,512 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 128)            │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 8)              │          1,032 │ dropout_3[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,837,960 (18.46 MB)

 Trainable params: 643,592 (2.46 MB)

 Non-trainable params: 4,194,368 (16.00 MB)

Epoch 1/50


I0000 00:00:1754826116.005744     101 cuda_dnn.cc:529] Loaded cuDNN version 90300


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 44s 33ms/step - accuracy: 0.6180 - loss: 0.9704 - precision: 0.9215 - recall: 0.4773 - val_accuracy: 0.0624 - val_loss: 6.5639 - val_precision: 0.2262 - val_recall: 0.0624 - learning_rate: 0.0010
Epoch 2/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 32s 32ms/step - accuracy: 0.7025 - loss: 0.7163 - precision: 0.9836 - recall: 0.5771 - val_accuracy: 0.0624 - val_loss: 8.5452 - val_precision: 0.2110 - val_recall: 0.0622 - learning_rate: 0.0010
Epoch 3/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 32s 32ms/step - accuracy: 0.7046 - loss: 0.7158 - precision: 0.9831 - recall: 0.5752 - val_accuracy: 0.0622 - val_loss: 7.8534 - val_precision: 0.2895 - val_recall: 0.0622 - learning_rate: 0.0010
Epoch 4/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 32s 32ms/step - accuracy: 0.7040 - loss: 0.7050 - precision: 0.9854 - recall: 0.5759 - val_accuracy: 0.0624 - val_loss: 9.6164 - val_precision: 0.2077 - val_recall: 0.0622 - learning_rate: 0.0010
Epoch 5/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 33s 33ms/step - a

{'acc': 0.581,
 'weighted_pre': 0.684585045622129,
 'weighted_rec': 0.581,
 'weighted_f1': 0.5593845756262176,
 'cm': [[4495, 502, 0, 0, 1, 1, 1, 0],
  [16, 4962, 2, 0, 20, 0, 0, 0],
  [16, 2643, 2327, 0, 14, 0, 0, 0],
  [2, 88, 0, 4909, 0, 1, 0, 0],
  [4, 3447, 5, 0, 1544, 0, 0, 0],
  [236, 618, 0, 0, 0, 4144, 2, 0],
  [7, 4105, 3, 0, 23, 3, 859, 0],
  [96, 3075, 8, 9, 18, 1162, 632, 0]]}